<a href="https://colab.research.google.com/github/vidyacheekuri/concrete-strength-prediction/blob/main/catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 13.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.feature_selection import SelectFromModel, RFE, mutual_info_regression
from sklearn.svm import SVR
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
import copy
from copy import deepcopy
import json
import optuna
import logging
import joblib
from pathlib import Path
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from scipy.stats import ttest_ind
warnings.filterwarnings('ignore')
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

In [3]:
# First check your numpy version
import numpy as np
print(np.version.version)

# Then reinstall CatBoost
!pip uninstall -y catboost
!pip install catboost

2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00


In [4]:
class EnhancedCatBoostPredictor:
    """Advanced predictor with deeper CatBoost, strength-specific models, and non-linear ensemble."""

    def __init__(self, random_state=42):
        self.random_state = random_state
        self.setup_logging()
        np.random.seed(random_state)

    def setup_logging(self):
        """Set up logging for the class."""
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        file_handler = logging.FileHandler('enhanced_catboost_predictor.log')
        file_handler.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)

    def engineer_features(self, X):
        """Create domain-specific engineered features for concrete strength prediction."""
        # Create a copy of the original dataframe
        X_engineered = X.copy()

        # Extract component names for readability
        cement = X['Cement (component 1)(kg in a m^3 mixture)']
        blast_slag = X['Blast Furnace Slag (component 2)(kg in a m^3 mixture)']
        fly_ash = X['Fly Ash (component 3)(kg in a m^3 mixture)']
        water = X['Water  (component 4)(kg in a m^3 mixture)']
        superplast = X['Superplasticizer (component 5)(kg in a m^3 mixture)']
        coarse_agg = X['Coarse Aggregate  (component 6)(kg in a m^3 mixture)']
        fine_agg = X['Fine Aggregate (component 7)(kg in a m^3 mixture)']
        age = X['Age (day)']

        # 1. Key concrete engineering ratios
        X_engineered['water_cement_ratio'] = water / (cement + 1e-5)
        X_engineered['total_cementitious'] = cement + blast_slag + fly_ash
        X_engineered['water_cementitious_ratio'] = water / (X_engineered['total_cementitious'] + 1e-5)
        X_engineered['agg_cement_ratio'] = (coarse_agg + fine_agg) / (cement + 1e-5)
        X_engineered['fine_coarse_ratio'] = fine_agg / (coarse_agg + 1e-5)

        # 2. Advanced cement chemistry features
        X_engineered['cementitious_superplast_ratio'] = X_engineered['total_cementitious'] / (superplast + 1e-5)
        X_engineered['cement_binder_ratio'] = cement / (X_engineered['total_cementitious'] + 1e-5)

        # 3. Time-dependent features
        X_engineered['log_age'] = np.log1p(age)
        X_engineered['sqrt_age'] = np.sqrt(age)
        X_engineered['age_28d_ratio'] = age / 28.0  # Normalization by standard 28-day strength

        # 4. Physical parameter approximations
        # Paste volume approximation (key determinant of strength)
        X_engineered['paste_volume'] = (cement / 3.15 + blast_slag / 2.9 + fly_ash / 2.3 + water) / \
                                      ((cement / 3.15 + blast_slag / 2.9 + fly_ash / 2.3 + water +
                                       coarse_agg / 2.7 + fine_agg / 2.6) + 1e-5)

        # 5. Practical concrete mix indicators
        X_engineered['slump_indicator'] = water + 10 * superplast
        X_engineered['flow_indicator'] = X_engineered['slump_indicator'] / X_engineered['total_cementitious']

        # 6. Concrete maturity index (time-temperature factor approximation)
        # Using age as simplified version (normally would include temperature)
        X_engineered['maturity_index'] = age * (1 - np.exp(-0.05 * age))

        # 7. Supplementary material utilization
        X_engineered['supplementary_fraction'] = (blast_slag + fly_ash) / (X_engineered['total_cementitious'] + 1e-5)

        # Enhanced age-related features, especially for very low strength range
        X_engineered['early_age_factor'] = np.where(X_engineered['Age (day)'] < 7,
                                                (7 - X_engineered['Age (day)'])/7, 0)
        X_engineered['very_early_strength'] = X_engineered['Age (day)']**0.5 * X_engineered['Cement (component 1)(kg in a m^3 mixture)']

        # Early hydration rate approximation (important for young concrete)
        X_engineered['early_hydration_rate'] = np.where(
            X_engineered['Age (day)'] < 7,
            X_engineered['Cement (component 1)(kg in a m^3 mixture)'] / (X_engineered['Age (day)'] + 0.5),
            0
        )

        # Late-age strength gain factor
        X_engineered['late_age_factor'] = np.where(
            X_engineered['Age (day)'] > 28,
            np.log1p(X_engineered['Age (day)'] - 28) / 4,
            0
        )

        # For very_low range (to help with over-prediction)
        X_engineered['very_low_correction'] = np.where(
            X_engineered['total_cementitious'] < X_engineered['total_cementitious'].mean(),
            -0.05 * X_engineered['water_cementitious_ratio'],
            0
        )

        # For high range (to help with under-prediction)
        X_engineered['high_correction'] = np.where(
            X_engineered['total_cementitious'] > X_engineered['total_cementitious'].mean() * 1.2,
            0.05 * X_engineered['cement_binder_ratio'],
            0
        )


        # Feature to detect abnormal mix designs that lead to prediction errors
        X_engineered['abnormal_mix_factor'] = np.abs(
            (X_engineered['water_cement_ratio'] - X_engineered['water_cement_ratio'].mean()) /
            X_engineered['water_cement_ratio'].std()
        )

        # Specialized feature for medium strength correction
        X_engineered['medium_correction'] = np.where(
            (X_engineered['total_cementitious'] >= 350) &
            (X_engineered['total_cementitious'] <= 450) &
            (X_engineered['water_cement_ratio'] <= 0.5),
            -0.1 * X_engineered['total_cementitious'],
            0
        )

        # Feature specifically for very low strength concrete with high water content
        X_engineered['water_excess_indicator'] = np.where(
            X_engineered['water_cement_ratio'] > 0.6,
            X_engineered['water_cement_ratio'] - 0.6,
            0
        )
        # Store feature information
        self.original_features = X.columns.tolist()
        self.engineered_features = [col for col in X_engineered.columns if col not in self.original_features]

        return X_engineered

    def load_and_preprocess(self, filepath):
        """Load data and preprocess with enhanced feature engineering."""
        try:
            self.data = pd.read_excel(filepath)
            self.logger.info("Data loaded successfully")

            # Split features and target
            X = self.data.drop(columns=['Concrete compressive strength(MPa, megapascals) '])
            y = self.data['Concrete compressive strength(MPa, megapascals) ']

            # Create engineered features
            X_engineered = self.engineer_features(X)
            self.logger.info(f"Created {len(self.engineered_features)} new engineered features")

            # Create strength ranges for stratified sampling and range-specific models
            strength_bins = [0, 20, 40, 60, 100]
            strength_labels = ['very_low', 'low', 'medium', 'high']
            y_ranges = pd.cut(y, bins=strength_bins, labels=strength_labels)
            self.y_ranges = y_ranges
            self.strength_bins = strength_bins
            self.strength_labels = strength_labels

            # Scale features
            self.scaler = StandardScaler()
            X_scaled = pd.DataFrame(
                self.scaler.fit_transform(X_engineered),
                columns=X_engineered.columns
            )
            X_scaled = X_scaled.reset_index(drop=True)

            # Store all features
            self.all_features = X_scaled.columns.tolist()

            # Split data with stratification by strength ranges
            X_train, X_test, y_train, y_test, y_ranges_train, y_ranges_test = train_test_split(
                X_scaled, y, y_ranges,
                test_size=0.2,
                random_state=self.random_state,
                stratify=y_ranges
            )
            X_train = X_train.reset_index(drop=True)
            X_test = X_test.reset_index(drop=True)

            self.X_train = X_train
            self.X_test = X_test
            self.y_train = y_train
            self.y_test = y_test
            self.y_ranges_train = y_ranges_train
            self.y_ranges_test = y_ranges_test

            print(f"Data split: {X_train.shape} training, {X_test.shape} testing")
            print("\nStrength range distribution in test set:")
            for label in strength_labels:
                count = np.sum(y_ranges_test == label)
                pct = count / len(y_ranges_test) * 100
                print(f"  {label.replace('_', ' ').title()}: {count} samples ({pct:.1f}%)")

            return X_train, X_test, y_train, y_test, y_ranges_train, y_ranges_test

        except Exception as e:
            self.logger.error(f"Error in preprocessing: {str(e)}")
            raise

    def train_deep_catboost(self):
        """Train a deeper CatBoost model with optimized parameters."""
        try:
            from catboost import CatBoostRegressor, Pool
            print("\nTraining deep CatBoost model...")

            # Create CatBoost model with deeper architecture
            deep_catboost = CatBoostRegressor(
                iterations=2000,          # Increased iterations
                learning_rate=0.02,       # Reduced learning rate
                depth=8,                  # Increased depth
                l2_leaf_reg=3,
                loss_function='RMSE',
                eval_metric='RMSE',
                random_seed=self.random_state,
                od_type='Iter',
                od_wait=100,              # More patience
                verbose=100,
                task_type='CPU',          # Use 'GPU' if available
                # Advanced parameters
                bootstrap_type='Bayesian',
                bagging_temperature=1,
                grow_policy='SymmetricTree',
                min_data_in_leaf=5
            )

            # Create train and eval pools
            train_pool = Pool(self.X_train, self.y_train)
            eval_pool = Pool(self.X_test, self.y_test)

            # Train model
            deep_catboost.fit(
                train_pool,
                eval_set=eval_pool,
                use_best_model=True,
                verbose=100
            )

            # Make predictions
            y_pred = deep_catboost.predict(self.X_test)

            # Calculate metrics
            metrics = self._calculate_metrics(self.y_test, y_pred)
            print("\nDeep CatBoost Model Metrics:")
            for metric, value in metrics.items():
                print(f"  {metric}: {value}")

            # Feature importance
            importance = deep_catboost.get_feature_importance()
            feature_importance = pd.DataFrame({
                'Feature': self.X_train.columns,
                'Importance': importance
            }).sort_values('Importance', ascending=False)

            print("\nTop 10 Features by Importance:")
            for idx, row in feature_importance.head(10).iterrows():
                print(f"  {row['Feature']}: {row['Importance']}")

            self.deep_catboost = deep_catboost
            self.catboost_feature_importance = feature_importance
            self.catboost_metrics = metrics
            self.catboost_preds = y_pred

            return metrics, y_pred

        except ImportError:
            print("CatBoost is not installed. Please install it using: pip install catboost")
            return None, None

    def train_range_specific_models(self):
        """Train separate models for different concrete strength ranges."""
        try:
            from catboost import CatBoostRegressor, Pool
            print("\nTraining strength range-specific models...")

            self.range_models = {}
            self.range_preds = {}

            # Updated parameters for different ranges with more focus on problematic ranges
            range_params = {
                'very_low': {  # Less than 20 MPa - Highest error rate
                    'iterations': 2000,        # Increased from 1000
                    'depth': 7,                # Increased from 6
                    'learning_rate': 0.02,     # Lower for more stability
                    'l2_leaf_reg': 5,          # Increased regularization
                    'bootstrap_type': 'Bayesian',
                    'min_data_in_leaf': 5,     # Increased to prevent overfitting
                    'random_strength': 0.9     # Increased randomization
                },
                'low': {  # 20-40 MPa
                    'iterations': 1500,
                    'depth': 7,
                    'learning_rate': 0.02,
                    'l2_leaf_reg': 3,
                    'bootstrap_type': 'Bayesian'
                },
                'medium': {  # 40-60 MPa
                    'iterations': 1500,
                    'depth': 8,
                    'learning_rate': 0.02,
                    'l2_leaf_reg': 3
                },
                'high': {  # Over 60 MPa - Few samples but high error rate
                    'iterations': 1200,        # Increased from 1000
                    'depth': 7,                # Increased from 6
                    'learning_rate': 0.015,    # Lower for stability
                    'l2_leaf_reg': 4,
                    'bootstrap_type': 'Bayesian',
                    'bagging_temperature': 1.5 # More aggressive bagging for few samples
                }
            }

            # Train separate model for each strength range
            for strength_range in self.strength_labels:
                print(f"\nTraining model for {strength_range.replace('_', ' ').title()} Strength range...")

                # Make sure indices are aligned properly - convert to numpy arrays if needed
                y_ranges_train_array = np.array(self.y_ranges_train)
                train_mask = (y_ranges_train_array == strength_range)

                # Check if we have enough samples
                if np.sum(train_mask) < 10:
                    print(f"  Not enough samples for {strength_range} range. Skipping.")
                    continue

                # Use .loc with indices to avoid alignment issues
                train_indices = np.where(train_mask)[0]
                X_train_range = self.X_train.iloc[train_indices]
                y_train_range = self.y_train.iloc[train_indices]

                # Similarly for test data
                y_ranges_test_array = np.array(self.y_ranges_test)
                test_mask = (y_ranges_test_array == strength_range)
                test_indices = np.where(test_mask)[0]

                if len(test_indices) < 5:
                    print(f"  Not enough test samples for {strength_range} range. Skipping metrics calculation.")
                    test_samples = 0
                else:
                    X_test_range = self.X_test.iloc[test_indices]
                    y_test_range = self.y_test.iloc[test_indices]
                    test_samples = len(X_test_range)

                print(f"  Training samples: {len(X_train_range)}, Test samples: {test_samples}")

                # Get parameters for this range
                model_params = range_params.get(strength_range, range_params['low'])  # Default to low params if not found

                # Create and train model with range-specific parameters
                range_model = CatBoostRegressor(
                    iterations=model_params['iterations'],
                    learning_rate=model_params['learning_rate'],
                    depth=model_params['depth'],
                    l2_leaf_reg=model_params.get('l2_leaf_reg', 3),
                    loss_function='RMSE',
                    eval_metric='RMSE',
                    random_seed=self.random_state,
                    od_type='Iter',
                    od_wait=50,
                    verbose=100,
                    bootstrap_type=model_params.get('bootstrap_type', 'Bayesian'),
                    min_data_in_leaf=model_params.get('min_data_in_leaf', 5),
                    random_strength=model_params.get('random_strength', 0.5),
                    bagging_temperature=model_params.get('bagging_temperature', 1.0)
                )

                # Create train pool
                train_pool = Pool(X_train_range, y_train_range)

                # Create eval pool if we have enough test samples
                if test_samples >= 5:
                    eval_pool = Pool(X_test_range, y_test_range)

                    # Train model with eval set
                    range_model.fit(
                        train_pool,
                        eval_set=eval_pool,
                        use_best_model=True,
                        verbose=100
                    )

                    # Calculate metrics
                    y_pred_range = range_model.predict(X_test_range)
                    metrics = self._calculate_metrics(y_test_range, y_pred_range)

                    print(f"  {strength_range.replace('_', ' ').title()} Range Model Metrics:")
                    for metric, value in metrics.items():
                        print(f"    {metric}: {value}")
                else:
                    # Train model without eval set
                    range_model.fit(
                        train_pool,
                        verbose=100
                    )

                # Store model
                self.range_models[strength_range] = range_model

                # Make predictions on full test set (for blending later)
                self.range_preds[strength_range] = range_model.predict(self.X_test)

            return self.range_models, self.range_preds

        except Exception as e:
            print(f"Error in training range-specific models: {str(e)}")
            import traceback
            traceback.print_exc()
            return None, None

    def train_very_low_specialized_models(self):
        """Train ultra-specialized models for very low strength concrete."""
        try:
            from catboost import CatBoostRegressor, Pool
            print("\nTraining specialized models for very low strength concrete...")

            # Get only very low samples using numpy arrays to avoid indexing issues
            y_ranges_train_array = np.array(self.y_ranges_train)
            mask = (y_ranges_train_array == 'very_low')

            # Check if we have enough samples
            if np.sum(mask) < 10:
                print("  Not enough very low strength samples. Skipping.")
                return {}, {}

            # Use indices instead of boolean masks
            train_indices = np.where(mask)[0]
            X_very_low = self.X_train.iloc[train_indices]
            y_very_low = self.y_train.iloc[train_indices]

            # Further split by actual strength for more specialization
            y_very_low_array = np.array(y_very_low)
            low_mask = y_very_low_array < 15  # Ultra-low strength
            mid_mask = (y_very_low_array >= 15) & (y_very_low_array < 20)  # Mid-low strength

            self.very_low_specialized_models = {}
            self.very_low_specialized_preds = {}

            # Ultra-low strength model
            if np.sum(low_mask) >= 10:
                # Get indices for the ultra-low samples
                ultra_low_indices = np.where(low_mask)[0]

                print(f"  Training ultra-low strength model (<15 MPa) with {len(ultra_low_indices)} samples")
                ultra_low_model = CatBoostRegressor(
                    iterations=1500,
                    depth=5,  # Lower depth to prevent overfitting on small samples
                    learning_rate=0.01,  # Lower learning rate for stability
                    l2_leaf_reg=6,  # Higher regularization
                    min_data_in_leaf=3,
                    verbose=0,
                    random_seed=self.random_state
                )

                # Select rows using iloc with indices
                X_ultra_low = X_very_low.iloc[ultra_low_indices]
                y_ultra_low = y_very_low.iloc[ultra_low_indices]

                ultra_low_model.fit(X_ultra_low, y_ultra_low)
                self.very_low_specialized_models['ultra_low'] = ultra_low_model

                # Make predictions on test set
                self.very_low_specialized_preds['ultra_low'] = np.zeros(len(self.X_test))

                # Identify test samples that would use this model
                # - First get very_low test samples
                y_ranges_test_array = np.array(self.y_ranges_test)
                test_mask = (y_ranges_test_array == 'very_low')
                test_indices = np.where(test_mask)[0]

                # - Then identify which ones are <15 MPa
                deep_preds = self.deep_catboost.predict(self.X_test)
                ultra_low_test_mask = (deep_preds < 15)

                # - Find intersection of very_low and <15 MPa
                X_test_very_low = self.X_test.iloc[test_indices]
                deep_preds_very_low = deep_preds[test_indices]
                ultra_low_test_indices = np.where(deep_preds_very_low < 15)[0]

                if len(ultra_low_test_indices) > 0:
                    # Calculate metrics
                    X_test_ultra_low = X_test_very_low.iloc[ultra_low_test_indices]
                    y_test_ultra_low = self.y_test.iloc[test_indices].iloc[ultra_low_test_indices]

                    ultra_low_preds = ultra_low_model.predict(X_test_ultra_low)
                    metrics = self._calculate_metrics(y_test_ultra_low, ultra_low_preds)

                    print(f"  Ultra-Low Strength Model Metrics (test samples: {len(ultra_low_test_indices)}):")
                    for metric, value in metrics.items():
                        print(f"    {metric}: {value}")

                    # Store predictions for meta-learner - using all test indices
                    for idx, very_low_idx in enumerate(test_indices):
                        if idx in ultra_low_test_indices:
                            test_sample = self.X_test.iloc[[very_low_idx]]
                            self.very_low_specialized_preds['ultra_low'][very_low_idx] = ultra_low_model.predict(test_sample)[0]

            # Mid-low strength model
            if np.sum(mid_mask) >= 10:
                # Get indices for the mid-low samples
                mid_low_indices = np.where(mid_mask)[0]

                print(f"  Training mid-low strength model (15-20 MPa) with {len(mid_low_indices)} samples")
                mid_low_model = CatBoostRegressor(
                    iterations=1500,
                    depth=6,
                    learning_rate=0.015,
                    l2_leaf_reg=4,
                    min_data_in_leaf=3,
                    verbose=0,
                    random_seed=self.random_state
                )

                # Select rows using iloc with indices
                X_mid_low = X_very_low.iloc[mid_low_indices]
                y_mid_low = y_very_low.iloc[mid_low_indices]

                mid_low_model.fit(X_mid_low, y_mid_low)
                self.very_low_specialized_models['mid_low'] = mid_low_model

                # Make predictions on test set
                self.very_low_specialized_preds['mid_low'] = np.zeros(len(self.X_test))

                # Identify test samples that would use this model
                # - First get very_low test samples
                y_ranges_test_array = np.array(self.y_ranges_test)
                test_mask = (y_ranges_test_array == 'very_low')
                test_indices = np.where(test_mask)[0]

                # - Then identify which ones are 15-20 MPa
                deep_preds = self.deep_catboost.predict(self.X_test)

                # - Find intersection of very_low and 15-20 MPa
                X_test_very_low = self.X_test.iloc[test_indices]
                deep_preds_very_low = deep_preds[test_indices]
                mid_low_test_indices = np.where((deep_preds_very_low >= 15) & (deep_preds_very_low < 20))[0]

                if len(mid_low_test_indices) > 0:
                    # Calculate metrics
                    X_test_mid_low = X_test_very_low.iloc[mid_low_test_indices]
                    y_test_mid_low = self.y_test.iloc[test_indices].iloc[mid_low_test_indices]

                    mid_low_preds = mid_low_model.predict(X_test_mid_low)
                    metrics = self._calculate_metrics(y_test_mid_low, mid_low_preds)

                    print(f"  Mid-Low Strength Model Metrics (test samples: {len(mid_low_test_indices)}):")
                    for metric, value in metrics.items():
                        print(f"    {metric}: {value}")

                    # Store predictions for meta-learner - using all test indices
                    for idx, very_low_idx in enumerate(test_indices):
                        if idx in mid_low_test_indices:
                            test_sample = self.X_test.iloc[[very_low_idx]]
                            self.very_low_specialized_preds['mid_low'][very_low_idx] = mid_low_model.predict(test_sample)[0]

            return self.very_low_specialized_models, self.very_low_specialized_preds

        except Exception as e:
            print(f"Error in training very low specialized models: {str(e)}")
            import traceback
            traceback.print_exc()
            return {}, {}

    def train_medium_bias_correction(self):
        """Create a bias correction model specifically for medium range."""
        try:
            from catboost import CatBoostRegressor
            print("\nTraining medium range bias correction model...")

            # Identify medium range samples using numpy arrays
            y_ranges_train_array = np.array(self.y_ranges_train)
            mask = (y_ranges_train_array == 'medium')

            # Get indices from mask
            train_indices = np.where(mask)[0]

            if len(train_indices) < 20:
                print("  Not enough medium range samples for bias correction. Skipping.")
                return None, None

            # Use indices to select rows
            X_medium = self.X_train.iloc[train_indices]
            y_medium = self.y_train.iloc[train_indices]

            # Calculate how much our main model over-predicts
            main_preds = self.deep_catboost.predict(X_medium)
            bias = main_preds - y_medium

            print(f"  Average bias in medium range: {bias.mean():.2f} MPa")
            print(f"  Max bias in medium range: {bias.max():.2f} MPa")

            # Train a model to predict this bias
            bias_model = CatBoostRegressor(
                iterations=800,
                depth=4,
                learning_rate=0.01,
                l2_leaf_reg=5,
                verbose=0,
                random_seed=self.random_state
            )

            bias_model.fit(X_medium, bias)
            self.medium_bias_model = bias_model

            # Make predictions on medium range test samples
            y_ranges_test_array = np.array(self.y_ranges_test)
            medium_test_mask = (y_ranges_test_array == 'medium')
            test_indices = np.where(medium_test_mask)[0]

            if len(test_indices) > 0:
                X_test_medium = self.X_test.iloc[test_indices]
                y_test_medium = self.y_test.iloc[test_indices]

                # Get the deep model predictions
                deep_preds_medium = self.deep_catboost.predict(X_test_medium)

                # Get the estimated bias
                estimated_bias = self.medium_bias_model.predict(X_test_medium)

                # Apply bias correction
                corrected_preds = deep_preds_medium - estimated_bias * 0.7  # 70% of the bias

                # Calculate metrics
                uncorrected_metrics = self._calculate_metrics(y_test_medium, deep_preds_medium)
                corrected_metrics = self._calculate_metrics(y_test_medium, corrected_preds)

                print("\n  Medium Range Before Correction:")
                for metric, value in uncorrected_metrics.items():
                    print(f"    {metric}: {value}")

                print("\n  Medium Range After Correction:")
                for metric, value in corrected_metrics.items():
                    print(f"    {metric}: {value}")

                # Store the bias predictions for meta-learner
                self.medium_bias_preds = np.zeros(len(self.X_test))
                for i, idx in enumerate(test_indices):
                    self.medium_bias_preds[idx] = estimated_bias[i]

            return self.medium_bias_model, getattr(self, 'medium_bias_preds', None)

        except Exception as e:
            print(f"Error in training medium bias correction: {str(e)}")
            import traceback
            traceback.print_exc()
            return None, None

    def train_boundary_models(self):
        """Train specialized models for boundary regions between strength ranges."""
        try:
            from catboost import CatBoostRegressor, Pool
            print("\nTraining boundary region models...")

            self.boundary_models = {}
            self.boundary_preds = {}

            # Define boundary regions with 2 MPa overlap on each side
            boundary_regions = [
                (15, 25, 'very_low_low_boundary'),  # Between very_low and low
                (38, 42, 'low_medium_boundary'),    # Between low and medium
                (58, 62, 'medium_high_boundary')    # Between medium and high
            ]

            for low_bound, high_bound, name in boundary_regions:
                print(f"\nTraining model for {name.replace('_', ' ').title()} region...")

                # Use numpy arrays to avoid indexing issues
                y_train_array = np.array(self.y_train)
                mask = (y_train_array >= low_bound) & (y_train_array <= high_bound)

                # Check if we have enough samples
                sample_count = np.sum(mask)

                if sample_count < 20:  # Skip if too few samples
                    print(f"  Insufficient samples ({sample_count}) for {name}. Skipping.")
                    continue

                # Use indices from the mask - this avoids pandas alignment issues
                train_indices = np.where(mask)[0]
                X_boundary = self.X_train.iloc[train_indices]
                y_boundary = self.y_train.iloc[train_indices]

                print(f"  Training with {len(X_boundary)} boundary samples.")

                # Create boundary-specific model
                boundary_model = CatBoostRegressor(
                    iterations=1200,
                    depth=6,
                    learning_rate=0.02,
                    l2_leaf_reg=3.5,
                    loss_function='RMSE',
                    eval_metric='RMSE',
                    random_seed=self.random_state,
                    od_type='Iter',
                    od_wait=50,
                    verbose=0
                )

                # Train model
                train_pool = Pool(X_boundary, y_boundary)
                boundary_model.fit(train_pool, verbose=100)

                # Store model
                self.boundary_models[name] = boundary_model

                # Make predictions on full test set (for blending later)
                self.boundary_preds[name] = boundary_model.predict(self.X_test)

                # Calculate metrics for boundary region test samples
                y_test_array = np.array(self.y_test)
                test_mask = (y_test_array >= low_bound) & (y_test_array <= high_bound)
                test_indices = np.where(test_mask)[0]

                if len(test_indices) > 0:
                    X_test_boundary = self.X_test.iloc[test_indices]
                    y_test_boundary = self.y_test.iloc[test_indices]

                    boundary_preds = boundary_model.predict(X_test_boundary)
                    metrics = self._calculate_metrics(y_test_boundary, boundary_preds)

                    print(f"  {name.replace('_', ' ').title()} Model Metrics:")
                    for metric, value in metrics.items():
                        print(f"    {metric}: {value}")

            return self.boundary_models, self.boundary_preds

        except Exception as e:
            print(f"Error in training boundary models: {str(e)}")
            import traceback
            traceback.print_exc()
            return None, None

    def train_age_specific_models(self):
        """Train specialized models for different concrete age groups."""
        try:
            from catboost import CatBoostRegressor, Pool
            print("\nTraining age-specific models...")

            self.age_models = {}
            self.age_preds = {}

            # Define age bins and labels
            age_bins = [0, 3, 7, 28, 90, float('inf')]
            age_labels = ['very_early', 'early', 'standard', 'mature', 'old']

            # Create age groups
            age_col = 'Age (day)'
            X_train_age = np.array(self.X_train[age_col])

            for i,age_group in enumerate(age_labels):
                if i >= len(age_bins) - 1:
                    continue  # Skip if we've reached the end of bins

                print(f"\nTraining model for {age_group.replace('_', ' ').title()} Age concrete...")

                # Get data for this age group using numpy for mask creation
                if i == len(age_bins) - 2:  # Last group
                    mask = (X_train_age >= age_bins[i]) & (X_train_age <= age_bins[i+1])
                else:
                    mask = (X_train_age >= age_bins[i]) & (X_train_age < age_bins[i+1])

                # Get indices from mask
                train_indices = np.where(mask)[0]
                sample_count = len(train_indices)

                if sample_count < 20:  # Skip if too few samples
                    print(f"  Insufficient samples ({sample_count}) for {age_group} age. Skipping.")
                    continue

                # Use indices to select rows
                X_age = self.X_train.iloc[train_indices]
                y_age = self.y_train.iloc[train_indices]

                print(f"  Training with {len(X_age)} age-specific samples.")

                # Create age-specific model with appropriate parameters
                if age_group in ['very_early', 'early']:
                    # More careful tuning for early-age concrete
                    age_model = CatBoostRegressor(
                        iterations=1500,
                        depth=6,
                        learning_rate=0.02,
                        l2_leaf_reg=4,
                        loss_function='RMSE',
                        eval_metric='RMSE',
                        random_seed=self.random_state,
                        od_type='Iter',
                        od_wait=50,
                        verbose=0
                    )
                else:
                    age_model = CatBoostRegressor(
                        iterations=1200,
                        depth=6,
                        learning_rate=0.025,
                        l2_leaf_reg=3,
                        loss_function='RMSE',
                        eval_metric='RMSE',
                        random_seed=self.random_state,
                        od_type='Iter',
                        od_wait=50,
                        verbose=0
                    )

                # Train model
                train_pool = Pool(X_age, y_age)
                age_model.fit(train_pool, verbose=100)

                # Store model
                self.age_models[age_group] = age_model

                # Make predictions on full test set (for blending later)
                self.age_preds[age_group] = age_model.predict(self.X_test)

                # Calculate metrics for age group test samples
                X_test_age = np.array(self.X_test[age_col])
                if i == len(age_bins) - 2:  # Last group
                    test_mask = (X_test_age >= age_bins[i]) & (X_test_age <= age_bins[i+1])
                else:
                    test_mask = (X_test_age >= age_bins[i]) & (X_test_age < age_bins[i+1])

                test_indices = np.where(test_mask)[0]

                if len(test_indices) > 0:
                    X_test_age_subset = self.X_test.iloc[test_indices]
                    y_test_age = self.y_test.iloc[test_indices]

                    age_preds = age_model.predict(X_test_age_subset)
                    metrics = self._calculate_metrics(y_test_age, age_preds)

                    print(f"  {age_group.replace('_', ' ').title()} Age Model Metrics:")
                    for metric, value in metrics.items():
                        print(f"    {metric}: {value}")

            return self.age_models, self.age_preds

        except ImportError:
            print("CatBoost is not installed. Please install it using: pip install catboost")
            return None, None

    def train_meta_learner(self):
        """Train a non-linear meta-learner with all specialized models."""
        # Check if we have the necessary models
        if not hasattr(self, 'deep_catboost'):
            print("Must train deep_catboost first!")
            return None, None

        print("\nTraining enhanced non-linear meta-learner ensemble...")

        # Create meta-features from all model predictions
        meta_features = [self.catboost_preds]  # Deep CatBoost predictions

        # Add range-specific model predictions
        for range_name in self.strength_labels:
            if range_name in self.range_preds:
                meta_features.append(self.range_preds[range_name])

        # Add boundary model predictions if available
        if hasattr(self, 'boundary_models') and self.boundary_models:
            for boundary_name in self.boundary_models:
                meta_features.append(self.boundary_preds[boundary_name])
                print(f"  Added {boundary_name} model predictions to meta-features")

        # Add age-specific model predictions if available
        if hasattr(self, 'age_models') and self.age_models:
            for age_group in self.age_models:
                meta_features.append(self.age_preds[age_group])
                print(f"  Added {age_group} age model predictions to meta-features")

        # Add very low specialized model predictions if available
        if hasattr(self, 'very_low_specialized_models') and self.very_low_specialized_models:
            for model_name in self.very_low_specialized_models:
                meta_features.append(self.very_low_specialized_preds[model_name])
                print(f"  Added very low {model_name} model predictions to meta-features")

        # Add medium bias model predictions if available
        if hasattr(self, 'medium_bias_model'):
            # Create a bias-corrected prediction vector
            bias_corrected_preds = self.catboost_preds.copy()
            medium_mask = (self.y_ranges_test == 'medium')

            # Convert medium_mask to numpy array for integer indexing
            medium_mask_array = medium_mask.to_numpy()

            for i in range(len(bias_corrected_preds)):
                if i < len(medium_mask_array) and medium_mask_array[i]:
                    bias = self.medium_bias_preds[i]
                    bias_corrected_preds[i] -= bias * 0.7

            meta_features.append(bias_corrected_preds)
            print(f"  Added medium bias-corrected predictions to meta-features")


        # Stack all meta-features
        meta_features_array = np.column_stack(meta_features)
        print(f"Meta-features shape: {meta_features_array.shape}")

        self.meta_feature_names = [f"meta_{i}" for i in range(meta_features_array.shape[1])]
        meta_features_df = pd.DataFrame(meta_features_array, columns=self.meta_feature_names)

        # Get strength range indicators (one-hot encoded)
        range_indicators = pd.get_dummies(self.y_ranges_test).values

        # Add range indicators and original features to meta-features
        meta_features_array = np.column_stack([
            meta_features_array,
            range_indicators,
            self.X_test.values  # Add original features for context
        ])

        # Try different meta-learners:
        # 1. CatBoost as meta-learner
        from catboost import CatBoostRegressor, Pool
        meta_catboost = CatBoostRegressor(
            iterations=1000,          # Increased iterations
            learning_rate=0.015,      # Reduced for stability
            depth=5,
            loss_function='RMSE',
            random_seed=self.random_state,
            verbose=0,
            l2_leaf_reg=4,            # Increased regularization
            bootstrap_type='Bayesian',
            grow_policy='SymmetricTree',
            min_data_in_leaf=5
        )

        # Split meta-features for training and validation
        meta_X_train, meta_X_val, meta_y_train, meta_y_val = train_test_split(
            meta_features_df, self.y_test,
            test_size=0.3,
            random_state=self.random_state
        )

        # Train meta-learner
        meta_catboost.fit(
            meta_X_train, meta_y_train,
            eval_set=(meta_X_val, meta_y_val),
            verbose=False
        )

        # Make meta-predictions
        meta_preds = meta_catboost.predict(meta_features_array)

        # Apply outlier detection and correction
        if hasattr(self, 'detect_and_correct_outliers'):
            meta_preds = self.detect_and_correct_outliers(self.X_test, meta_preds)

        # Calculate metrics
        metrics = self._calculate_metrics(self.y_test, meta_preds)
        print("\nMeta-Learner (CatBoost) Metrics:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value}")

        # 2. Try MLP as meta-learner
        from sklearn.neural_network import MLPRegressor

        # Normalize meta-features for MLP
        meta_features_scaled = StandardScaler().fit_transform(meta_features_array)

        meta_mlp = MLPRegressor(
            hidden_layer_sizes=(64, 32, 16),  # Larger network
            activation='relu',
            solver='adam',
            alpha=0.001,
            batch_size='auto',
            learning_rate='adaptive',
            max_iter=2000,              # Increased
            early_stopping=True,
            validation_fraction=0.2,
            n_iter_no_change=20,
            random_state=self.random_state
        )

        # Split meta-features for training and validation
        meta_X_train_scaled, meta_X_val_scaled, meta_y_train, meta_y_val = train_test_split(
            meta_features_scaled, self.y_test,
            test_size=0.3,
            random_state=self.random_state
        )

        # Train MLP meta-learner
        meta_mlp.fit(meta_X_train_scaled, meta_y_train)

        # Make MLP meta-predictions
        mlp_meta_preds = meta_mlp.predict(meta_features_scaled)

        # Apply outlier detection and correction
        if hasattr(self, 'detect_and_correct_outliers'):
            mlp_meta_preds = self.detect_and_correct_outliers(self.X_test, mlp_meta_preds)

        # Calculate metrics
        mlp_metrics = self._calculate_metrics(self.y_test, mlp_meta_preds)
        print("\nMeta-Learner (MLP) Metrics:")
        for metric, value in mlp_metrics.items():
            print(f"  {metric}: {value}")

        # 3. Try stacking with weighted average
        print("\nTrying weighted ensemble of meta-learners...")

        # Optimize weights using a simple grid search
        best_rmse = float('inf')
        best_weights = (0.5, 0.5)

        for catboost_weight in np.linspace(0.0, 1.0, 11):
            mlp_weight = 1.0 - catboost_weight

            # Combine predictions
            weighted_preds = (catboost_weight * meta_preds) + (mlp_weight * mlp_meta_preds)

            # Calculate RMSE
            rmse = np.sqrt(mean_squared_error(self.y_test, weighted_preds))

            if rmse < best_rmse:
                best_rmse = rmse
                best_weights = (catboost_weight, mlp_weight)

       # Decide the best meta-learner based on metrics (e.g., percent within 10%)
        best_metric = max(
            metrics['percent_within_10'],
            mlp_metrics['percent_within_10']
        )

        if best_metric == mlp_metrics['percent_within_10']:
            print("\nUsing MLP as meta-learner (best performance)")
            self.meta_learner = meta_mlp
            self.meta_learner_type = 'mlp'
            self.meta_features_scaler = StandardScaler().fit(meta_features_array)
        else:
            print("\nUsing CatBoost as meta-learner (best performance)")
            self.meta_learner = meta_catboost
            self.meta_learner_type = 'catboost'
            self.meta_features_scaler = None

        # Save both models and weights anyway (for ensemble fallback)
        self.meta_catboost = meta_catboost
        self.meta_mlp = meta_mlp
        self.meta_weights = best_weights
        self.meta_preds = meta_preds  # optional if used later


        print(f"Best weights: CatBoost={best_weights[0]:.2f}, MLP={best_weights[1]:.2f}")

        # Create final ensemble predictions
        ensemble_preds = (best_weights[0] * meta_preds) + (best_weights[1] * mlp_meta_preds)

        # Apply outlier detection and correction
        if hasattr(self, 'detect_and_correct_outliers'):
            ensemble_preds = self.detect_and_correct_outliers(self.X_test, ensemble_preds)

        # Calculate metrics
        ensemble_metrics = self._calculate_metrics(self.y_test, ensemble_preds)
        print("\nEnsemble Meta-Learner Metrics:")
        for metric, value in ensemble_metrics.items():
            print(f"  {metric}: {value}")

        # 4. Apply range-specific corrections
        corrected_preds = ensemble_preds.copy()

        for i, pred in enumerate(corrected_preds):
            # Determine strength range based on predicted value
            if pred < 20:
                strength_range = 'very_low'
            elif pred < 40:
                strength_range = 'low'
            elif pred < 60:
                strength_range = 'medium'
            else:
                strength_range = 'high'

            # Apply specialized corrections
            if strength_range == 'very_low':
                # Check for specialized very low models
                if hasattr(self, 'very_low_specialized_models'):
                    if pred < 15 and 'ultra_low' in self.very_low_specialized_models:
                        # Get specialized prediction
                        specialized_pred = self.very_low_specialized_models['ultra_low'].predict([self.X_test.iloc[i]])[0]
                        # Use a weighted blend
                        corrected_preds[i] = 0.4 * pred + 0.6 * specialized_pred
                    elif pred >= 15 and pred < 20 and 'mid_low' in self.very_low_specialized_models:
                        specialized_pred = self.very_low_specialized_models['mid_low'].predict([self.X_test.iloc[i]])[0]
                        corrected_preds[i] = 0.4 * pred + 0.6 * specialized_pred

            elif strength_range == 'medium':
                # Apply bias correction for medium range
                if hasattr(self, 'medium_bias_model'):
                    estimated_bias = self.medium_bias_model.predict([self.X_test.iloc[i]])[0]
                    # If bias is significant
                    if estimated_bias > 5:
                        # Reduce the prediction by the estimated bias
                        corrected_preds[i] -= estimated_bias * 0.7  # Using 70% of the bias as a safe measure

            elif strength_range == 'high':
                # Boost high strength predictions to address under-prediction
                corrected_preds[i] *= 1.05  # Apply a 5% boost

        # Calculate metrics for range-corrected ensemble
        corrected_metrics = self._calculate_metrics(self.y_test, corrected_preds)
        print("\nRange-Corrected Ensemble Meta-Learner Metrics:")
        for metric, value in corrected_metrics.items():
            print(f"  {metric}: {value}")

        # Choose the best meta-learner based on percent_within_10 metric
        best_metric = max(
            metrics['percent_within_10'],
            mlp_metrics['percent_within_10'],
            ensemble_metrics['percent_within_10'],
            corrected_metrics['percent_within_10']
        )

        if best_metric == corrected_metrics['percent_within_10']:
            print("\nUsing range-corrected ensemble as meta-learner (better performance)")
            self.meta_learner_type = 'corrected_ensemble'
            self.meta_catboost = meta_catboost
            self.meta_mlp = meta_mlp
            self.meta_weights = best_weights
            self.meta_features_scaler = StandardScaler().fit(meta_features_array)
            self.meta_preds = corrected_preds
            self.meta_metrics = corrected_metrics
        elif best_metric == ensemble_metrics['percent_within_10']:
            print("\nUsing weighted ensemble as meta-learner (better performance)")
            self.meta_learner_type = 'ensemble'
            self.meta_catboost = meta_catboost
            self.meta_mlp = meta_mlp
            self.meta_weights = best_weights
            self.meta_features_scaler = StandardScaler().fit(meta_features_array)
            self.meta_preds = ensemble_preds
            self.meta_metrics = ensemble_metrics
        elif best_metric == mlp_metrics['percent_within_10']:
            print("\nUsing MLP as meta-learner (better performance)")
            self.meta_learner = meta_mlp
            self.meta_learner_type = 'mlp'
            self.meta_features_scaler = StandardScaler().fit(meta_features_array)
            self.meta_preds = mlp_meta_preds
            self.meta_metrics = mlp_metrics
        else:
            print("\nUsing CatBoost as meta-learner")
            self.meta_learner = meta_catboost
            self.meta_learner_type = 'catboost'
            self.meta_features_scaler = None
            self.meta_preds = meta_preds
            self.meta_metrics = metrics

        # Create meta feature generation function for new data
        self._create_meta_feature_generator()

        # Analyze improvement by strength range
        print("\nImprovement Analysis by Strength Range:")
        print("-" * 70)

        original_errors = np.abs((self.y_test - self.catboost_preds) / self.y_test * 100)
        meta_errors = np.abs((self.y_test - self.meta_preds) / self.y_test * 100)

        for strength_range in self.strength_labels:
            range_mask = (self.y_ranges_test == strength_range)
            if np.sum(range_mask) > 0:
                original_within_10 = np.mean(original_errors[range_mask] <= 10) * 100
                meta_within_10 = np.mean(meta_errors[range_mask] <= 10) * 100
                improvement = meta_within_10 - original_within_10

                original_mean_error = np.mean(original_errors[range_mask])
                meta_mean_error = np.mean(meta_errors[range_mask])
                error_reduction = original_mean_error - meta_mean_error

                print(f"Strength Range: {strength_range.replace('_', ' ').title()}")
                print(f"  Original within 10%: {original_within_10:.2f}%")
                print(f"  Meta-learner within 10%: {meta_within_10:.2f}%")
                print(f"  Improvement: {improvement:.2f} percentage points")
                print(f"  Mean error reduction: {error_reduction:.2f}%")
                print("-" * 70)

        return self.meta_metrics, self.meta_preds

    def _create_meta_feature_generator(self):
        """Create a function to generate meta-features for new data."""
        def generate_meta_features(self, X):
          """Generate meta-features for new data samples."""
          meta_features = []

          # Deep CatBoost predictions
          deep_preds = self.deep_catboost.predict(X)
          meta_features.append(deep_preds)

          # Range-specific models
          for range_name in self.strength_labels:
              if hasattr(self, 'range_models') and range_name in self.range_models:
                  meta_features.append(self.range_models[range_name].predict(X))

          # Boundary models
          if hasattr(self, 'boundary_models') and self.boundary_models:
              for name, model in self.boundary_models.items():
                  meta_features.append(model.predict(X))

          # Age-specific models
          if hasattr(self, 'age_models') and self.age_models:
              for age_group, model in self.age_models.items():
                  meta_features.append(model.predict(X))

          # Very low models
          if hasattr(self, 'very_low_specialized_models') and self.very_low_specialized_models:
              for name, model in self.very_low_specialized_models.items():
                  meta_features.append(model.predict(X))

          # Bias-corrected predictions
          if hasattr(self, 'medium_bias_model'):
              bias_corrected_preds = deep_preds.copy()
              medium_mask = (deep_preds >= 40) & (deep_preds < 60)
              if np.any(medium_mask):
                  medium_indices = np.where(medium_mask)[0]
                  X_medium = X.iloc[medium_indices]
                  bias_predictions = self.medium_bias_model.predict(X_medium)
                  for idx, i in enumerate(medium_indices):
                      bias_corrected_preds[i] -= bias_predictions[idx] * 0.7
              meta_features.append(bias_corrected_preds)

          # Estimate range and create one-hot
          estimated_ranges = pd.cut(deep_preds, bins=self.strength_bins, labels=self.strength_labels)
          range_indicators = pd.get_dummies(estimated_ranges).reindex(columns=self.strength_labels, fill_value=0).values

          # Stack everything
          meta_features_array = np.column_stack(meta_features)
          meta_features_array = np.column_stack([meta_features_array, range_indicators, X.values])

          # Convert to DataFrame with proper column names
          meta_feature_names = [f"meta_{i}" for i in range(meta_features_array.shape[1])]
          meta_features_df = pd.DataFrame(meta_features_array, columns = self.meta_feature_names)


          print("✅ Final meta feature shape:", meta_features_array.shape)
          print("📦 CatBoost expects:", self.meta_catboost.feature_count_)

          return meta_features_df

        self.generate_meta_features = generate_meta_features

    def _calculate_metrics(self, y_true, y_pred):
        """Calculate comprehensive performance metrics."""
        # Calculate basic regression metrics
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

        # Calculate percentage errors
        percent_errors = np.abs((y_true - y_pred) / y_true * 100)

        return {
            'r2': r2,
            'rmse': rmse,
            'mae': mae,
            'max_percent_error': np.max(percent_errors),
            'mean_percent_error': np.mean(percent_errors),
            'median_percent_error': np.median(percent_errors),
            'percent_within_5': np.mean(percent_errors <= 5) * 100,
            'percent_within_10': np.mean(percent_errors <= 10) * 100
        }

    def save_model(self, filepath='models/enhanced_catboost_model.joblib'):
        """Save the trained models and preprocessing objects."""
        model_dir = Path('models')
        model_dir.mkdir(exist_ok=True)

        # Create dictionary with all model components
        model_data = {
            'deep_catboost': getattr(self, 'deep_catboost', None),
            'range_models': getattr(self, 'range_models', {}),
            'meta_learner': getattr(self, 'meta_learner', None),
            'meta_learner_type': getattr(self, 'meta_learner_type', None),
            'meta_features_scaler': getattr(self, 'meta_features_scaler', None),
            'meta_weights': getattr(self, 'meta_weights', None),
            'meta_catboost': getattr(self, 'meta_catboost', None),
            'meta_mlp': getattr(self, 'meta_mlp', None),
            'meta_feature_names': getattr(self, 'meta_feature_names', None),
            'scaler': self.scaler,
            'original_features': self.original_features,
            'engineered_features': self.engineered_features,
            'all_features': self.all_features,
            'strength_bins': self.strength_bins,
            'strength_labels': self.strength_labels,
            'random_state': self.random_state,
            'catboost_preds': getattr(self, 'catboost_preds', None),
            'meta_preds': getattr(self, 'meta_preds', None),
            'meta_metrics': getattr(self, 'meta_metrics', None)
        }

        joblib.dump(model_data, filepath)
        print(f"Enhanced CatBoost models saved to {filepath}")

    @classmethod
    def load_model(cls, filepath='models/enhanced_catboost_model.joblib'):
        """Load a trained model and preprocessing objects."""
        model_data = joblib.load(filepath)

        predictor = cls()

        if 'meta_feature_names' in model_data:
          predictor.meta_feature_names = model_data['meta_feature_names']

        for key, value in model_data.items():
            setattr(predictor, key, value)

        # Recreate meta-feature generator
        if hasattr(predictor, 'meta_learner'):
            predictor._create_meta_feature_generator()

        return predictor

    def detect_and_correct_outliers(self, X, predictions):
        """Detect and correct likely outlier predictions."""
        corrected_predictions = predictions.copy()

        # Get features that might indicate outlier behavior
        if 'water_cement_ratio' in X.columns and 'abnormal_mix_factor' in X.columns:
            wcr = X['water_cement_ratio']
            abnormal_factor = X['abnormal_mix_factor']

            # Identify potential outliers based on extreme ratios and factors
            wcr_array = np.array(wcr)
            abnormal_factor_array = np.array(abnormal_factor)
            wcr_high = wcr_array > np.quantile(wcr_array, 0.95)
            wcr_low = wcr_array < np.quantile(wcr_array, 0.05)
            abnormal_high = abnormal_factor_array > 2.0

            potential_outliers = wcr_high | wcr_low | abnormal_high

            # For these potential outliers, use a more conservative prediction
            outlier_indices = np.where(potential_outliers)[0]
            if len(outlier_indices) > 0:
                print(f"Detected {len(outlier_indices)} potential outlier predictions")

                for i in outlier_indices:
                    # Estimate strength range based on predicted value
                    pred_value = predictions[i]
                    if pred_value < 20:
                        strength_range = 'very_low'
                    elif pred_value < 40:
                        strength_range = 'low'
                    elif pred_value < 60:
                        strength_range = 'medium'
                    else:
                        strength_range = 'high'

                    # Use range-specific model if available
                    if hasattr(self, 'range_models') and strength_range in self.range_models:
                        # Use iloc with a list to access a single row as DataFrame
                        range_pred = self.range_models[strength_range].predict(X.iloc[[i]])[0]
                        # Use a weighted average with more weight on range model
                        corrected_predictions[i] = 0.3 * predictions[i] + 0.7 * range_pred
                        print(f"  Outlier at index {i}: Original {predictions[i]:.2f}, Corrected {corrected_predictions[i]:.2f}")

        return corrected_predictions

    def predict(self, X_new):
        """Make predictions with all specialized models and corrections."""
        if not hasattr(self, 'meta_learner'):
            raise ValueError("Meta-learner has not been trained. Call train_meta_learner first.")

        # Preprocess data
        if isinstance(X_new, pd.DataFrame):
            X_engineered = self.engineer_features(X_new)
        else:
            # Convert to DataFrame if numpy array
            X_new_df = pd.DataFrame(X_new, columns=self.original_features)
            X_engineered = self.engineer_features(X_new_df)

        # Scale features
        X_scaled = self.scaler.transform(X_engineered)
        X_scaled_df = pd.DataFrame(X_scaled, columns=self.all_features)

        # Generate meta-features
        meta_features = self.generate_meta_features(X_scaled_df)

        # Make predictions using meta-learner
        predictions = self.meta_learner.predict(meta_features)

        # Apply outlier detection and correction
        predictions = self.detect_and_correct_outliers(X_scaled_df, predictions)

        # Apply range-specific corrections
        final_predictions = []

        for i, pred in enumerate(predictions):
            # Determine likely strength range
            if pred < 20:
                strength_range = 'very_low'
            elif pred < 40:
                strength_range = 'low'
            elif pred < 60:
                strength_range = 'medium'
            else:
                strength_range = 'high'

            # Apply specialized corrections
            if strength_range == 'very_low':
                # Check for specialized very low models
                if hasattr(self, 'very_low_specialized_models'):
                    if pred < 15 and 'ultra_low' in self.very_low_specialized_models:
                        # Get specialized prediction
                        specialized_pred = self.very_low_specialized_models['ultra_low'].predict([X_scaled_df.iloc[i]])[0]
                        # Use a weighted blend
                        pred = 0.4 * pred + 0.6 * specialized_pred
                    elif pred >= 15 and pred < 20 and 'mid_low' in self.very_low_specialized_models:
                        specialized_pred = self.very_low_specialized_models['mid_low'].predict([X_scaled_df.iloc[i]])[0]
                        pred = 0.4 * pred + 0.6 * specialized_pred

            elif strength_range == 'medium':
                # Apply bias correction for medium range
                if hasattr(self, 'medium_bias_model'):
                    estimated_bias = self.medium_bias_model.predict([X_scaled_df.iloc[i]])[0]
                    # If bias is significant
                    if estimated_bias > 5:
                        # Reduce the prediction by the estimated bias
                        pred -= estimated_bias * 0.7  # Using 70% of the bias as a safe measure

            elif strength_range == 'high':
                # Boost high strength predictions to address under-prediction
                pred *= 1.05  # Apply a 5% boost

            final_predictions.append(pred)

        return np.array(final_predictions)

In [6]:
predictor = EnhancedCatBoostPredictor()
predictor.load_and_preprocess("Concrete_Data.xls")

INFO:__main__:Data loaded successfully
INFO:__main__:Created 24 new engineered features


Data split: (824, 32) training, (206, 32) testing

Strength range distribution in test set:
  Very Low: 39 samples (18.9%)
  Low: 91 samples (44.2%)
  Medium: 57 samples (27.7%)
  High: 19 samples (9.2%)


(     Cement (component 1)(kg in a m^3 mixture)  \
 0                                    -1.092950   
 1                                    -0.602506   
 2                                     2.477918   
 3                                     1.185513   
 4                                    -1.205916   
 ..                                         ...   
 819                                  -1.112097   
 820                                  -0.285245   
 821                                  -0.508974   
 822                                  -0.866253   
 823                                   1.070632   
 
      Blast Furnace Slag (component 2)(kg in a m^3 mixture)  \
 0                                             1.311551       
 1                                            -0.223285       
 2                                            -0.856886       
 3                                            -0.856886       
 4                                             1.275604       
 ..     

In [7]:
predictor.train_deep_catboost()


Training deep CatBoost model...
0:	learn: 16.3773542	test: 16.8433682	best: 16.8433682 (0)	total: 109ms	remaining: 3m 37s
100:	learn: 6.1136361	test: 6.8700188	best: 6.8700188 (100)	total: 1.89s	remaining: 35.4s
200:	learn: 4.2346431	test: 5.4830746	best: 5.4830746 (200)	total: 4.8s	remaining: 43s
300:	learn: 3.5294387	test: 5.0404466	best: 5.0404466 (300)	total: 6.84s	remaining: 38.6s
400:	learn: 3.0696553	test: 4.7563153	best: 4.7563153 (400)	total: 8.64s	remaining: 34.4s
500:	learn: 2.7190146	test: 4.5948980	best: 4.5948980 (500)	total: 10.4s	remaining: 31.2s
600:	learn: 2.4682995	test: 4.4794460	best: 4.4794460 (600)	total: 12.2s	remaining: 28.5s
700:	learn: 2.2491978	test: 4.3924141	best: 4.3924141 (700)	total: 14s	remaining: 26s
800:	learn: 2.0811114	test: 4.3347376	best: 4.3347376 (800)	total: 16.1s	remaining: 24.2s
900:	learn: 1.9414535	test: 4.2742268	best: 4.2742268 (900)	total: 19s	remaining: 23.2s
1000:	learn: 1.8189255	test: 4.2211912	best: 4.2211912 (1000)	total: 20.8s	r

({'r2': 0.9454105850797221,
  'rmse': np.float64(3.9876280695215987),
  'mae': 2.541734733167335,
  'max_percent_error': 53.14030601586066,
  'mean_percent_error': np.float64(8.018560171532583),
  'median_percent_error': np.float64(5.397953349717742),
  'percent_within_5': np.float64(48.05825242718447),
  'percent_within_10': np.float64(74.75728155339806)},
 array([43.61207026, 61.49479866, 46.46236698, 41.81398376, 21.52711967,
        44.55214236, 56.22356169, 38.22973886, 33.9837826 , 42.52655591,
        63.81638206, 14.76681534, 62.92449641, 27.42021999, 30.79300563,
        77.45458432, 16.05869803, 18.12391663, 42.73258308, 43.03899875,
        36.62586143, 38.33452519, 58.46214471, 29.76554483, 28.1527394 ,
        13.21921306, 15.67583963, 67.61644959, 33.39852324, 32.71487612,
        25.24862009, 50.50822315, 61.12417784, 13.40428437, 38.71290202,
        35.4706168 , 27.0072475 , 26.5537361 , 41.81510426, 59.33023374,
        54.69028735, 73.26758392, 33.63459459, 44.897592

In [8]:
predictor.train_range_specific_models()
predictor.train_boundary_models()
predictor.train_very_low_specialized_models()
predictor.train_medium_bias_correction()
predictor.train_age_specific_models()


Training strength range-specific models...

Training model for Very Low Strength range...
  Training samples: 158, Test samples: 39
0:	learn: 4.0389472	test: 3.3239354	best: 3.3239354 (0)	total: 4.46ms	remaining: 8.93s
100:	learn: 2.5743132	test: 2.5735631	best: 2.5735631 (100)	total: 385ms	remaining: 7.24s
200:	learn: 1.9464592	test: 2.2855583	best: 2.2855583 (200)	total: 767ms	remaining: 6.86s
300:	learn: 1.5616363	test: 2.1736765	best: 2.1736765 (300)	total: 1.16s	remaining: 6.52s
400:	learn: 1.2608670	test: 2.1383783	best: 2.1379128 (394)	total: 1.57s	remaining: 6.25s
500:	learn: 1.0307389	test: 2.1265864	best: 2.1240791 (487)	total: 1.96s	remaining: 5.87s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 2.122767836
bestIteration = 515

Shrink model to first 516 iterations.
  Very Low Range Model Metrics:
    r2: 0.5868293996228349
    rmse: 2.1227678884153742
    mae: 1.4854258388119068
    max_percent_error: 53.6679890667082
    mean_percent_error: 11.7284138256

({'very_early': <catboost.core.CatBoostRegressor at 0x7d7f1a0ae250>,
  'early': <catboost.core.CatBoostRegressor at 0x7d7f1a0dfe10>},
 {'very_early': array([44.52871852, 62.12585394, 60.08461518, 59.13993677, 34.54649853,
         56.19159082, 64.73370095, 36.04023783, 34.70299321, 57.62221303,
         63.58480311, 46.14274672, 56.54308138, 50.29182263, 43.78633423,
         77.39813034, 40.20330743, 43.32030008, 41.41909692, 43.56731323,
         64.11520274, 40.39365906, 55.66636755, 43.04023531, 47.08850446,
         35.06525884, 35.13139962, 68.12342066, 45.67014938, 37.88884326,
         43.64762853, 59.05386221, 59.30109362, 38.93077273, 46.20610357,
         47.97403637, 50.12560799, 57.21227035, 50.39598637, 61.71780982,
         55.23830371, 73.2900559 , 41.73546174, 74.56655603, 45.20749814,
         40.11467803, 39.71730269, 44.95314242, 44.28357857, 53.96178845,
         63.16277936, 36.21720915, 35.04997572, 39.74199138, 39.56260186,
         71.45727753, 50.76694476, 37.

In [9]:
predictor.train_meta_learner()


Training enhanced non-linear meta-learner ensemble...
  Added very_low_low_boundary model predictions to meta-features
  Added low_medium_boundary model predictions to meta-features
  Added medium_high_boundary model predictions to meta-features
  Added very_early age model predictions to meta-features
  Added early age model predictions to meta-features
  Added very low ultra_low model predictions to meta-features
  Added very low mid_low model predictions to meta-features
  Added medium bias-corrected predictions to meta-features
Meta-features shape: (206, 13)
Detected 22 potential outlier predictions
  Outlier at index 13: Original 29.35, Corrected 27.59
  Outlier at index 21: Original 39.22, Corrected 36.98
  Outlier at index 29: Original 30.90, Corrected 31.95
  Outlier at index 42: Original 34.57, Corrected 34.55
  Outlier at index 47: Original 28.79, Corrected 26.14
  Outlier at index 50: Original 41.18, Corrected 45.85
  Outlier at index 53: Original 9.20, Corrected 9.43
  Out

({'r2': 0.9642066451299548,
  'rmse': np.float64(3.22895228724342),
  'mae': 1.7973990661926504,
  'max_percent_error': 56.505090520010334,
  'mean_percent_error': np.float64(5.717630607942577),
  'median_percent_error': np.float64(3.380815369039619),
  'percent_within_5': np.float64(62.62135922330098),
  'percent_within_10': np.float64(83.49514563106796)},
 array([38.68874831, 60.04992413, 45.52201943, 38.55588437, 23.85364478,
        44.70940788, 60.73836673, 38.5626481 , 30.34930844, 44.4628984 ,
        64.25451151, 15.13712602, 59.57836448, 27.58641375, 31.12437303,
        74.8964845 , 13.39188389, 20.31933269, 44.10702636, 45.57821551,
        35.18500913, 36.98262308, 60.08666939, 32.20254078, 27.95368893,
        13.61314927, 15.17916236, 66.55634887, 25.88852579, 31.94706872,
        25.54940193, 53.24943198, 59.56942723, 16.71087349, 34.95351251,
        36.07962794, 24.72192583, 26.49302692, 38.99845483, 72.276022  ,
        53.47151813, 75.55228388, 34.55066871, 55.320740

In [10]:
predictor.save_model("models/enhanced_catboost_model.joblib")

Enhanced CatBoost models saved to models/enhanced_catboost_model.joblib


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

# Assuming EnhancedCatBoostPredictor class is defined in the same script or imported
# If EnhancedCatBoostPredictor is in a separate file named 'enhanced_catboost_predictor.py',
# you would uncomment the import below:
# from enhanced_catboost_predictor import EnhancedCatBoostPredictor

class EnhancedCatBoostPredictor:
    """Advanced predictor with deeper CatBoost, strength-specific models, and non-linear ensemble."""

    def __init__(self, random_state=42):
        self.random_state = random_state
        self.logger = None # Initialize logger here, setup_logging will configure it
        np.random.seed(random_state)

    def setup_logging(self):
        """Set up logging for the class."""
        import logging
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        # Prevent adding multiple handlers if already set up
        if not self.logger.handlers:
            file_handler = logging.FileHandler('enhanced_catboost_predictor.log')
            file_handler.setLevel(logging.INFO)
            formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
            file_handler.setFormatter(formatter)
            self.logger.addHandler(file_handler)

    def engineer_features(self, X):
        """Create domain-specific engineered features for concrete strength prediction."""
        X_engineered = X.copy()
        cement = X['Cement (component 1)(kg in a m^3 mixture)']
        blast_slag = X['Blast Furnace Slag (component 2)(kg in a m^3 mixture)']
        fly_ash = X['Fly Ash (component 3)(kg in a m^3 mixture)']
        water = X['Water  (component 4)(kg in a m^3 mixture)']
        superplast = X['Superplasticizer (component 5)(kg in a m^3 mixture)']
        coarse_agg = X['Coarse Aggregate  (component 6)(kg in a m^3 mixture)']
        fine_agg = X['Fine Aggregate (component 7)(kg in a m^3 mixture)']
        age = X['Age (day)']

        X_engineered['water_cement_ratio'] = water / (cement + 1e-5)
        X_engineered['total_cementitious'] = cement + blast_slag + fly_ash
        X_engineered['water_cementitious_ratio'] = water / (X_engineered['total_cementitious'] + 1e-5)
        X_engineered['agg_cement_ratio'] = (coarse_agg + fine_agg) / (cement + 1e-5)
        X_engineered['fine_coarse_ratio'] = fine_agg / (coarse_agg + 1e-5)
        X_engineered['cementitious_superplast_ratio'] = X_engineered['total_cementitious'] / (superplast + 1e-5)
        X_engineered['cement_binder_ratio'] = cement / (X_engineered['total_cementitious'] + 1e-5)
        X_engineered['log_age'] = np.log1p(age)
        X_engineered['sqrt_age'] = np.sqrt(age)
        X_engineered['age_28d_ratio'] = age / 28.0
        X_engineered['paste_volume'] = (cement / 3.15 + blast_slag / 2.9 + fly_ash / 2.3 + water) / \
                                      ((cement / 3.15 + blast_slag / 2.9 + fly_ash / 2.3 + water +
                                       coarse_agg / 2.7 + fine_agg / 2.6) + 1e-5)
        X_engineered['slump_indicator'] = water + 10 * superplast
        X_engineered['flow_indicator'] = X_engineered['slump_indicator'] / X_engineered['total_cementitious']
        X_engineered['maturity_index'] = age * (1 - np.exp(-0.05 * age))
        X_engineered['supplementary_fraction'] = (blast_slag + fly_ash) / (X_engineered['total_cementitious'] + 1e-5)
        X_engineered['early_age_factor'] = np.where(X_engineered['Age (day)'] < 7, (7 - X_engineered['Age (day)'])/7, 0)
        X_engineered['very_early_strength'] = X_engineered['Age (day)']**0.5 * X_engineered['Cement (component 1)(kg in a m^3 mixture)']
        X_engineered['early_hydration_rate'] = np.where(X_engineered['Age (day)'] < 7,
                                                        X_engineered['Cement (component 1)(kg in a m^3 mixture)'] / (X_engineered['Age (day)'] + 0.5), 0)
        X_engineered['late_age_factor'] = np.where(X_engineered['Age (day)'] > 28,
                                                   np.log1p(X_engineered['Age (day)'] - 28) / 4, 0)
        X_engineered['very_low_correction'] = np.where(X_engineered['total_cementitious'] < X_engineered['total_cementitious'].mean(),
                                                      -0.05 * X_engineered['water_cementitious_ratio'], 0)
        X_engineered['high_correction'] = np.where(X_engineered['total_cementitious'] > X_engineered['total_cementitious'].mean() * 1.2,
                                                  0.05 * X_engineered['cement_binder_ratio'], 0)
        X_engineered['abnormal_mix_factor'] = np.abs(
            (X_engineered['water_cement_ratio'] - X_engineered['water_cement_ratio'].mean()) /
            X_engineered['water_cement_ratio'].std()
        )
        X_engineered['medium_correction'] = np.where(
            (X_engineered['total_cementitious'] >= 350) &
            (X_engineered['total_cementitious'] <= 450) &
            (X_engineered['water_cement_ratio'] <= 0.5),
            -0.1 * X_engineered['total_cementitious'], 0
        )
        X_engineered['water_excess_indicator'] = np.where(X_engineered['water_cement_ratio'] > 0.6,
                                                         X_engineered['water_cement_ratio'] - 0.6, 0)
        self.original_features = X.columns.tolist()
        self.engineered_features = [col for col in X_engineered.columns if col not in self.original_features]
        return X_engineered

    def load_and_preprocess(self, filepath):
        """Load data and preprocess with enhanced feature engineering."""
        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import StandardScaler
        from scipy.stats import ttest_ind

        self.setup_logging()
        try:
            self.data = pd.read_excel(filepath)
            self.logger.info("Data loaded successfully")

            X = self.data.drop(columns=['Concrete compressive strength(MPa, megapascals) '])
            y = self.data['Concrete compressive strength(MPa, megapascals) ']

            X_engineered = self.engineer_features(X)
            self.logger.info(f"Created {len(self.engineered_features)} new engineered features")

            strength_bins = [0, 20, 40, 60, 100]
            strength_labels = ['very_low', 'low', 'medium', 'high']
            y_ranges = pd.cut(y, bins=strength_bins, labels=strength_labels)
            self.y_ranges = y_ranges
            self.strength_bins = strength_bins
            self.strength_labels = strength_labels

            self.scaler = StandardScaler()
            X_scaled = pd.DataFrame(
                self.scaler.fit_transform(X_engineered),
                columns=X_engineered.columns
            )
            X_scaled = X_scaled.reset_index(drop=True)

            self.all_features = X_scaled.columns.tolist()

            X_train, X_test, y_train, y_test, y_ranges_train, y_ranges_test = train_test_split(
                X_scaled, y, y_ranges,
                test_size=0.2,
                random_state=self.random_state,
                stratify=y_ranges
            )
            X_train = X_train.reset_index(drop=True)
            X_test = X_test.reset_index(drop=True)

            self.X_train = X_train
            self.X_test = X_test
            self.y_train = y_train
            self.y_test = y_test
            self.y_ranges_train = y_ranges_train
            self.y_ranges_test = y_ranges_test

            print(f"Data split: {X_train.shape} training, {X_test.shape} testing")
            print("\nStrength range distribution in test set:")
            for label in strength_labels:
                count = np.sum(y_ranges_test == label)
                pct = count / len(y_ranges_test) * 100
                print(f"  {label.replace('_', ' ').title()}: {count} samples ({pct:.1f}%)")

            return X_train, X_test, y_train, y_test, y_ranges_train, y_ranges_test

        except Exception as e:
            self.logger.error(f"Error in preprocessing: {str(e)}")
            raise

    def train_deep_catboost(self):
        from catboost import CatBoostRegressor, Pool
        print("\nTraining deep CatBoost model...")
        deep_catboost = CatBoostRegressor(
            iterations=2000, learning_rate=0.02, depth=8, l2_leaf_reg=3,
            loss_function='RMSE', eval_metric='RMSE', random_seed=self.random_state,
            od_type='Iter', od_wait=100, verbose=100, task_type='CPU',
            bootstrap_type='Bayesian', bagging_temperature=1, grow_policy='SymmetricTree',
            min_data_in_leaf=5
        )
        train_pool = Pool(self.X_train, self.y_train)
        eval_pool = Pool(self.X_test, self.y_test)
        deep_catboost.fit(train_pool, eval_set=eval_pool, use_best_model=True, verbose=100)
        y_pred = deep_catboost.predict(self.X_test)
        metrics = self._calculate_metrics(self.y_test, y_pred)
        print("\nDeep CatBoost Model Metrics:")
        for metric, value in metrics.items(): print(f"  {metric}: {value}")
        importance = deep_catboost.get_feature_importance()
        feature_importance = pd.DataFrame({'Feature': self.X_train.columns, 'Importance': importance}).sort_values('Importance', ascending=False)
        print("\nTop 10 Features by Importance:")
        for idx, row in feature_importance.head(10).iterrows(): print(f"  {row['Feature']}: {row['Importance']}")
        self.deep_catboost = deep_catboost
        self.catboost_feature_importance = feature_importance
        self.catboost_metrics = metrics
        self.catboost_preds = y_pred
        return metrics, y_pred

    def train_range_specific_models(self):
        from catboost import CatBoostRegressor, Pool
        print("\nTraining strength range-specific models...")
        self.range_models = {}
        self.range_preds = {}
        range_params = {
            'very_low': {'iterations': 2000, 'depth': 7, 'learning_rate': 0.02, 'l2_leaf_reg': 5, 'bootstrap_type': 'Bayesian', 'min_data_in_leaf': 5, 'random_strength': 0.9},
            'low': {'iterations': 1500, 'depth': 7, 'learning_rate': 0.02, 'l2_leaf_reg': 3, 'bootstrap_type': 'Bayesian'},
            'medium': {'iterations': 1500, 'depth': 8, 'learning_rate': 0.02, 'l2_leaf_reg': 3},
            'high': {'iterations': 1200, 'depth': 7, 'learning_rate': 0.015, 'l2_leaf_reg': 4, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.5}
        }
        for strength_range in self.strength_labels:
            print(f"\nTraining model for {strength_range.replace('_', ' ').title()} Strength range...")
            y_ranges_train_array = np.array(self.y_ranges_train)
            train_mask = (y_ranges_train_array == strength_range)
            if np.sum(train_mask) < 10:
                print(f"  Not enough samples for {strength_range} range. Skipping.")
                continue
            train_indices = np.where(train_mask)[0]
            X_train_range = self.X_train.iloc[train_indices]
            y_train_range = self.y_train.iloc[train_indices]
            y_ranges_test_array = np.array(self.y_ranges_test)
            test_mask = (y_ranges_test_array == strength_range)
            test_indices = np.where(test_mask)[0]
            if len(test_indices) < 5:
                print(f"  Not enough test samples for {strength_range} range. Skipping metrics calculation.")
                test_samples = 0
            else:
                X_test_range = self.X_test.iloc[test_indices]
                y_test_range = self.y_test.iloc[test_indices]
                test_samples = len(X_test_range)
            print(f"  Training samples: {len(X_train_range)}, Test samples: {test_samples}")
            model_params = range_params.get(strength_range, range_params['low'])
            range_model = CatBoostRegressor(
                iterations=model_params['iterations'], learning_rate=model_params['learning_rate'],
                depth=model_params['depth'], l2_leaf_reg=model_params.get('l2_leaf_reg', 3),
                loss_function='RMSE', eval_metric='RMSE', random_seed=self.random_state,
                od_type='Iter', od_wait=50, verbose=100, bootstrap_type=model_params.get('bootstrap_type', 'Bayesian'),
                min_data_in_leaf=model_params.get('min_data_in_leaf', 5), random_strength=model_params.get('random_strength', 0.5),
                bagging_temperature=model_params.get('bagging_temperature', 1.0)
            )
            train_pool = Pool(X_train_range, y_train_range)
            if test_samples >= 5:
                eval_pool = Pool(X_test_range, y_test_range)
                range_model.fit(train_pool, eval_set=eval_pool, use_best_model=True, verbose=100)
                y_pred_range = range_model.predict(X_test_range)
                metrics = self._calculate_metrics(y_test_range, y_pred_range)
                print(f"  {strength_range.replace('_', ' ').title()} Range Model Metrics:")
                for metric, value in metrics.items(): print(f"    {metric}: {value}")
            else:
                range_model.fit(train_pool, verbose=100)
            self.range_models[strength_range] = range_model
            self.range_preds[strength_range] = range_model.predict(self.X_test)
        return self.range_models, self.range_preds

    def train_very_low_specialized_models(self):
        from catboost import CatBoostRegressor, Pool
        print("\nTraining specialized models for very low strength concrete...")
        y_ranges_train_array = np.array(self.y_ranges_train)
        mask = (y_ranges_train_array == 'very_low')
        if np.sum(mask) < 10:
            print("  Not enough very low strength samples. Skipping.")
            return {}, {}
        train_indices = np.where(mask)[0]
        X_very_low = self.X_train.iloc[train_indices]
        y_very_low = self.y_train.iloc[train_indices]
        y_very_low_array = np.array(y_very_low)
        low_mask = y_very_low_array < 15
        mid_mask = (y_very_low_array >= 15) & (y_very_low_array < 20)
        self.very_low_specialized_models = {}
        self.very_low_specialized_preds = {}
        if np.sum(low_mask) >= 10:
            ultra_low_indices = np.where(low_mask)[0]
            print(f"  Training ultra-low strength model (<15 MPa) with {len(ultra_low_indices)} samples")
            ultra_low_model = CatBoostRegressor(iterations=1500, depth=5, learning_rate=0.01, l2_leaf_reg=6, min_data_in_leaf=3, verbose=0, random_seed=self.random_state)
            X_ultra_low = X_very_low.iloc[ultra_low_indices]
            y_ultra_low = y_very_low.iloc[ultra_low_indices]
            ultra_low_model.fit(X_ultra_low, y_ultra_low)
            self.very_low_specialized_models['ultra_low'] = ultra_low_model
            self.very_low_specialized_preds['ultra_low'] = np.zeros(len(self.X_test))
            y_ranges_test_array = np.array(self.y_ranges_test)
            test_mask = (y_ranges_test_array == 'very_low')
            test_indices = np.where(test_mask)[0]
            deep_preds = self.deep_catboost.predict(self.X_test)
            deep_preds_very_low = deep_preds[test_indices]
            ultra_low_test_indices = np.where(deep_preds_very_low < 15)[0]
            if len(ultra_low_test_indices) > 0:
                X_test_very_low = self.X_test.iloc[test_indices]
                X_test_ultra_low = X_test_very_low.iloc[ultra_low_test_indices]
                y_test_ultra_low = self.y_test.iloc[test_indices].iloc[ultra_low_test_indices]
                ultra_low_preds = ultra_low_model.predict(X_test_ultra_low)
                metrics = self._calculate_metrics(y_test_ultra_low, ultra_low_preds)
                print(f"  Ultra-Low Strength Model Metrics (test samples: {len(ultra_low_test_indices)}):")
                for metric, value in metrics.items(): print(f"    {metric}: {value}")
                for idx, very_low_idx in enumerate(test_indices):
                    if idx in ultra_low_test_indices:
                        test_sample = self.X_test.iloc[[very_low_idx]]
                        self.very_low_specialized_preds['ultra_low'][very_low_idx] = ultra_low_model.predict(test_sample)[0]
        if np.sum(mid_mask) >= 10:
            mid_low_indices = np.where(mid_mask)[0]
            print(f"  Training mid-low strength model (15-20 MPa) with {len(mid_low_indices)} samples")
            mid_low_model = CatBoostRegressor(iterations=1500, depth=6, learning_rate=0.015, l2_leaf_reg=4, min_data_in_leaf=3, verbose=0, random_seed=self.random_state)
            X_mid_low = X_very_low.iloc[mid_low_indices]
            y_mid_low = y_very_low.iloc[mid_low_indices]
            mid_low_model.fit(X_mid_low, y_mid_low)
            self.very_low_specialized_models['mid_low'] = mid_low_model
            self.very_low_specialized_preds['mid_low'] = np.zeros(len(self.X_test))
            y_ranges_test_array = np.array(self.y_ranges_test)
            test_mask = (y_ranges_test_array == 'very_low')
            test_indices = np.where(test_mask)[0]
            deep_preds = self.deep_catboost.predict(self.X_test)
            X_test_very_low = self.X_test.iloc[test_indices]
            deep_preds_very_low = deep_preds[test_indices]
            mid_low_test_indices = np.where((deep_preds_very_low >= 15) & (deep_preds_very_low < 20))[0]
            if len(mid_low_test_indices) > 0:
                X_test_mid_low = X_test_very_low.iloc[mid_low_test_indices]
                y_test_mid_low = self.y_test.iloc[test_indices].iloc[mid_low_test_indices]
                mid_low_preds = mid_low_model.predict(X_test_mid_low)
                metrics = self._calculate_metrics(y_test_mid_low, mid_low_preds)
                print(f"  Mid-Low Strength Model Metrics (test samples: {len(mid_low_test_indices)}):")
                for metric, value in metrics.items(): print(f"    {metric}: {value}")
                for idx, very_low_idx in enumerate(test_indices):
                    if idx in mid_low_test_indices:
                        test_sample = self.X_test.iloc[[very_low_idx]]
                        self.very_low_specialized_preds['mid_low'][very_low_idx] = mid_low_model.predict(test_sample)[0]
        return self.very_low_specialized_models, self.very_low_specialized_preds

    def train_medium_bias_correction(self):
        from catboost import CatBoostRegressor
        print("\nTraining medium range bias correction model...")
        y_ranges_train_array = np.array(self.y_ranges_train)
        mask = (y_ranges_train_array == 'medium')
        train_indices = np.where(mask)[0]
        if len(train_indices) < 20:
            print("  Not enough medium range samples for bias correction. Skipping.")
            return None, None
        X_medium = self.X_train.iloc[train_indices]
        y_medium = self.y_train.iloc[train_indices]
        main_preds = self.deep_catboost.predict(X_medium)
        bias = main_preds - y_medium
        print(f"  Average bias in medium range: {bias.mean():.2f} MPa")
        print(f"  Max bias in medium range: {bias.max():.2f} MPa")
        bias_model = CatBoostRegressor(iterations=800, depth=4, learning_rate=0.01, l2_leaf_reg=5, verbose=0, random_seed=self.random_state)
        bias_model.fit(X_medium, bias)
        self.medium_bias_model = bias_model
        y_ranges_test_array = np.array(self.y_ranges_test)
        medium_test_mask = (y_ranges_test_array == 'medium')
        test_indices = np.where(medium_test_mask)[0]
        if len(test_indices) > 0:
            X_test_medium = self.X_test.iloc[test_indices]
            y_test_medium = self.y_test.iloc[test_indices]
            deep_preds_medium = self.deep_catboost.predict(X_test_medium)
            estimated_bias = self.medium_bias_model.predict(X_test_medium)
            corrected_preds = deep_preds_medium - estimated_bias * 0.7
            uncorrected_metrics = self._calculate_metrics(y_test_medium, deep_preds_medium)
            corrected_metrics = self._calculate_metrics(y_test_medium, corrected_preds)
            print("\n  Medium Range Before Correction:")
            for metric, value in uncorrected_metrics.items(): print(f"    {metric}: {value}")
            print("\n  Medium Range After Correction:")
            for metric, value in corrected_metrics.items(): print(f"    {metric}: {value}")
            self.medium_bias_preds = np.zeros(len(self.X_test))
            for i, idx in enumerate(test_indices): self.medium_bias_preds[idx] = estimated_bias[i]
        return self.medium_bias_model, getattr(self, 'medium_bias_preds', None)

    def train_boundary_models(self):
        from catboost import CatBoostRegressor, Pool
        print("\nTraining boundary region models...")
        self.boundary_models = {}
        self.boundary_preds = {}
        boundary_regions = [(15, 25, 'very_low_low_boundary'), (38, 42, 'low_medium_boundary'), (58, 62, 'medium_high_boundary')]
        for low_bound, high_bound, name in boundary_regions:
            print(f"\nTraining model for {name.replace('_', ' ').title()} region...")
            y_train_array = np.array(self.y_train)
            mask = (y_train_array >= low_bound) & (y_train_array <= high_bound)
            sample_count = np.sum(mask)
            if sample_count < 20:
                print(f"  Insufficient samples ({sample_count}) for {name}. Skipping.")
                continue
            train_indices = np.where(mask)[0]
            X_boundary = self.X_train.iloc[train_indices]
            y_boundary = self.y_train.iloc[train_indices]
            print(f"  Training with {len(X_boundary)} boundary samples.")
            boundary_model = CatBoostRegressor(
                iterations=1200, depth=6, learning_rate=0.02, l2_leaf_reg=3.5,
                loss_function='RMSE', eval_metric='RMSE', random_seed=self.random_state,
                od_type='Iter', od_wait=50, verbose=0
            )
            train_pool = Pool(X_boundary, y_boundary)
            boundary_model.fit(train_pool, verbose=100)
            self.boundary_models[name] = boundary_model
            self.boundary_preds[name] = boundary_model.predict(self.X_test)
            y_test_array = np.array(self.y_test)
            test_mask = (y_test_array >= low_bound) & (y_test_array <= high_bound)
            test_indices = np.where(test_mask)[0]
            if len(test_indices) > 0:
                X_test_boundary = self.X_test.iloc[test_indices]
                y_test_boundary = self.y_test.iloc[test_indices]
                boundary_preds = boundary_model.predict(X_test_boundary)
                metrics = self._calculate_metrics(y_test_boundary, boundary_preds)
                print(f"  {name.replace('_', ' ').title()} Model Metrics:")
                for metric, value in metrics.items(): print(f"    {metric}: {value}")
        return self.boundary_models, self.boundary_preds

    def train_age_specific_models(self):
        from catboost import CatBoostRegressor, Pool
        print("\nTraining age-specific models...")
        self.age_models = {}
        self.age_preds = {}
        age_bins = [0, 3, 7, 28, 90, float('inf')]
        age_labels = ['very_early', 'early', 'standard', 'mature', 'old']
        age_col = 'Age (day)'
        X_train_age = np.array(self.X_train[age_col])
        for i,age_group in enumerate(age_labels):
            if i >= len(age_bins) - 1: continue
            print(f"\nTraining model for {age_group.replace('_', ' ').title()} Age concrete...")
            if i == len(age_bins) - 2: mask = (X_train_age >= age_bins[i]) & (X_train_age <= age_bins[i+1])
            else: mask = (X_train_age >= age_bins[i]) & (X_train_age < age_bins[i+1])
            train_indices = np.where(mask)[0]
            sample_count = len(train_indices)
            if sample_count < 20:
                print(f"  Insufficient samples ({sample_count}) for {age_group} age. Skipping.")
                continue
            X_age = self.X_train.iloc[train_indices]
            y_age = self.y_train.iloc[train_indices]
            print(f"  Training with {len(X_age)} age-specific samples.")
            if age_group in ['very_early', 'early']:
                age_model = CatBoostRegressor(iterations=1500, depth=6, learning_rate=0.02, l2_leaf_reg=4, loss_function='RMSE', eval_metric='RMSE', random_seed=self.random_state, od_type='Iter', od_wait=50, verbose=0)
            else:
                age_model = CatBoostRegressor(iterations=1200, depth=6, learning_rate=0.025, l2_leaf_reg=3, loss_function='RMSE', eval_metric='RMSE', random_seed=self.random_state, od_type='Iter', od_wait=50, verbose=0)
            train_pool = Pool(X_age, y_age)
            age_model.fit(train_pool, verbose=100)
            self.age_models[age_group] = age_model
            self.age_preds[age_group] = age_model.predict(self.X_test)
            X_test_age = np.array(self.X_test[age_col])
            if i == len(age_bins) - 2: test_mask = (X_test_age >= age_bins[i]) & (X_test_age <= age_bins[i+1])
            else: test_mask = (X_test_age >= age_bins[i]) & (X_test_age < age_bins[i+1])
            test_indices = np.where(test_mask)[0]
            if len(test_indices) > 0:
                X_test_age_subset = self.X_test.iloc[test_indices]
                y_test_age = self.y_test.iloc[test_indices]
                age_preds = age_model.predict(X_test_age_subset)
                metrics = self._calculate_metrics(y_test_age, age_preds)
                print(f"  {age_group.replace('_', ' ').title()} Age Model Metrics:")
                for metric, value in metrics.items(): print(f"    {metric}: {value}")
        return self.age_models, self.age_preds

    def train_meta_learner(self):
        if not hasattr(self, 'deep_catboost'):
            print("Must train deep_catboost first!")
            return None, None
        print("\nTraining enhanced non-linear meta-learner ensemble...")
        meta_features = [self.catboost_preds]
        for range_name in self.strength_labels:
            if range_name in self.range_preds:
                meta_features.append(self.range_preds[range_name])
        if hasattr(self, 'boundary_models') and self.boundary_models:
            for boundary_name in self.boundary_models:
                meta_features.append(self.boundary_preds[boundary_name])
                print(f"  Added {boundary_name} model predictions to meta-features")
        if hasattr(self, 'age_models') and self.age_models:
            for age_group in self.age_models:
                meta_features.append(self.age_preds[age_group])
                print(f"  Added {age_group} age model predictions to meta-features")
        if hasattr(self, 'very_low_specialized_models') and self.very_low_specialized_models:
            for model_name in self.very_low_specialized_models:
                meta_features.append(self.very_low_specialized_preds[model_name])
                print(f"  Added very low {model_name} model predictions to meta-features")
        if hasattr(self, 'medium_bias_model'):
            bias_corrected_preds = self.catboost_preds.copy()
            medium_mask = (self.y_ranges_test == 'medium')
            medium_mask_array = medium_mask.to_numpy()
            for i in range(len(bias_corrected_preds)):
                if i < len(medium_mask_array) and medium_mask_array[i]:
                    bias = self.medium_bias_preds[i]
                    bias_corrected_preds[i] -= bias * 0.7
            meta_features.append(bias_corrected_preds)
            print(f"  Added medium bias-corrected predictions to meta-features")

        meta_features_array = np.column_stack(meta_features)
        print(f"Meta-features shape: {meta_features_array.shape}")
        self.meta_feature_names = [f"meta_{i}" for i in range(meta_features_array.shape[1])]
        meta_features_df = pd.DataFrame(meta_features_array, columns=self.meta_feature_names)

        range_indicators = pd.get_dummies(self.y_ranges_test).values
        meta_features_array = np.column_stack([meta_features_array, range_indicators, self.X_test.values])

        from catboost import CatBoostRegressor, Pool
        meta_catboost = CatBoostRegressor(
            iterations=1000, learning_rate=0.015, depth=5, loss_function='RMSE',
            random_seed=self.random_state, verbose=0, l2_leaf_reg=4,
            bootstrap_type='Bayesian', grow_policy='SymmetricTree', min_data_in_leaf=5
        )
        meta_X_train, meta_X_val, meta_y_train, meta_y_val = train_test_split(
            meta_features_df, self.y_test, test_size=0.3, random_state=self.random_state
        )
        meta_catboost.fit(meta_X_train, meta_y_train, eval_set=(meta_X_val, meta_y_val), verbose=False)
        meta_preds = meta_catboost.predict(meta_features_array)
        if hasattr(self, 'detect_and_correct_outliers'):
            meta_preds = self.detect_and_correct_outliers(self.X_test, meta_preds)
        metrics = self._calculate_metrics(self.y_test, meta_preds)
        print("\nMeta-Learner (CatBoost) Metrics:")
        for metric, value in metrics.items(): print(f"  {metric}: {value}")

        from sklearn.neural_network import MLPRegressor
        meta_features_scaled = StandardScaler().fit_transform(meta_features_array)
        meta_mlp = MLPRegressor(
            hidden_layer_sizes=(64, 32, 16), activation='relu', solver='adam', alpha=0.001,
            batch_size='auto', learning_rate='adaptive', max_iter=2000, early_stopping=True,
            validation_fraction=0.2, n_iter_no_change=20, random_state=self.random_state
        )
        meta_X_train_scaled, meta_X_val_scaled, meta_y_train, meta_y_val = train_test_split(
            meta_features_scaled, self.y_test, test_size=0.3, random_state=self.random_state
        )
        meta_mlp.fit(meta_X_train_scaled, meta_y_train)
        mlp_meta_preds = meta_mlp.predict(meta_features_scaled)
        if hasattr(self, 'detect_and_correct_outliers'):
            mlp_meta_preds = self.detect_and_correct_outliers(self.X_test, mlp_meta_preds)
        mlp_metrics = self._calculate_metrics(self.y_test, mlp_meta_preds)
        print("\nMeta-Learner (MLP) Metrics:")
        for metric, value in mlp_metrics.items(): print(f"  {metric}: {value}")

        print("\nTrying weighted ensemble of meta-learners...")
        best_rmse = float('inf')
        best_weights = (0.5, 0.5)
        for catboost_weight in np.linspace(0.0, 1.0, 11):
            mlp_weight = 1.0 - catboost_weight
            weighted_preds = (catboost_weight * meta_preds) + (mlp_weight * mlp_meta_preds)
            rmse = np.sqrt(mean_squared_error(self.y_test, weighted_preds))
            if rmse < best_rmse:
                best_rmse = rmse
                best_weights = (catboost_weight, mlp_weight)

        best_metric = max(metrics['percent_within_10'], mlp_metrics['percent_within_10'])
        if best_metric == mlp_metrics['percent_within_10']:
            print("\nUsing MLP as meta-learner (best performance)")
            self.meta_learner = meta_mlp
            self.meta_learner_type = 'mlp'
            self.meta_features_scaler = StandardScaler().fit(meta_features_array)
        else:
            print("\nUsing CatBoost as meta-learner (best performance)")
            self.meta_learner = meta_catboost
            self.meta_learner_type = 'catboost'
            self.meta_features_scaler = None

        self.meta_catboost = meta_catboost
        self.meta_mlp = meta_mlp
        self.meta_weights = best_weights
        self.meta_preds = meta_preds

        print(f"Best weights: CatBoost={best_weights[0]:.2f}, MLP={best_weights[1]:.2f}")
        ensemble_preds = (best_weights[0] * meta_preds) + (best_weights[1] * mlp_meta_preds)
        if hasattr(self, 'detect_and_correct_outliers'):
            ensemble_preds = self.detect_and_correct_outliers(self.X_test, ensemble_preds)
        ensemble_metrics = self._calculate_metrics(self.y_test, ensemble_preds)
        print("\nEnsemble Meta-Learner Metrics:")
        for metric, value in ensemble_metrics.items(): print(f"  {metric}: {value}")

        corrected_preds = ensemble_preds.copy()
        for i, pred in enumerate(corrected_preds):
            if pred < 20: strength_range = 'very_low'
            elif pred < 40: strength_range = 'low'
            elif pred < 60: strength_range = 'medium'
            else: strength_range = 'high'
            if strength_range == 'very_low':
                if hasattr(self, 'very_low_specialized_models'):
                    if pred < 15 and 'ultra_low' in self.very_low_specialized_models:
                        specialized_pred = self.very_low_specialized_models['ultra_low'].predict([self.X_test.iloc[i]])[0]
                        corrected_preds[i] = 0.4 * pred + 0.6 * specialized_pred
                    elif pred >= 15 and pred < 20 and 'mid_low' in self.very_low_specialized_models:
                        specialized_pred = self.very_low_specialized_models['mid_low'].predict([self.X_test.iloc[i]])[0]
                        corrected_preds[i] = 0.4 * pred + 0.6 * specialized_pred
            elif strength_range == 'medium':
                if hasattr(self, 'medium_bias_model'):
                    estimated_bias = self.medium_bias_model.predict([self.X_test.iloc[i]])[0]
                    if estimated_bias > 5: corrected_preds[i] -= estimated_bias * 0.7
            elif strength_range == 'high': corrected_preds[i] *= 1.05

        corrected_metrics = self._calculate_metrics(self.y_test, corrected_preds)
        print("\nRange-Corrected Ensemble Meta-Learner Metrics:")
        for metric, value in corrected_metrics.items(): print(f"  {metric}: {value}")

        best_metric = max(metrics['percent_within_10'], mlp_metrics['percent_within_10'], ensemble_metrics['percent_within_10'], corrected_metrics['percent_within_10'])
        if best_metric == corrected_metrics['percent_within_10']:
            print("\nUsing range-corrected ensemble as meta-learner (better performance)")
            self.meta_learner_type = 'corrected_ensemble'
            self.meta_catboost = meta_catboost
            self.meta_mlp = meta_mlp
            self.meta_weights = best_weights
            self.meta_features_scaler = StandardScaler().fit(meta_features_array)
            self.meta_preds = corrected_preds
            self.meta_metrics = corrected_metrics
        elif best_metric == ensemble_metrics['percent_within_10']:
            print("\nUsing weighted ensemble as meta-learner (better performance)")
            self.meta_learner_type = 'ensemble'
            self.meta_catboost = meta_catboost
            self.meta_mlp = meta_mlp
            self.meta_weights = best_weights
            self.meta_features_scaler = StandardScaler().fit(meta_features_array)
            self.meta_preds = ensemble_preds
            self.meta_metrics = ensemble_metrics
        elif best_metric == mlp_metrics['percent_within_10']:
            print("\nUsing MLP as meta-learner (better performance)")
            self.meta_learner = meta_mlp
            self.meta_learner_type = 'mlp'
            self.meta_features_scaler = StandardScaler().fit(meta_features_array)
            self.meta_preds = mlp_meta_preds
            self.meta_metrics = mlp_metrics
        else:
            print("\nUsing CatBoost as meta-learner")
            self.meta_learner = meta_catboost
            self.meta_learner_type = 'catboost'
            self.meta_features_scaler = None
            self.meta_preds = meta_preds
            self.meta_metrics = metrics

        self._create_meta_feature_generator()
        print("\nImprovement Analysis by Strength Range:")
        print("-" * 70)
        original_errors = np.abs((self.y_test - self.catboost_preds) / self.y_test * 100)
        meta_errors = np.abs((self.y_test - self.meta_preds) / self.y_test * 100)
        for strength_range in self.strength_labels:
            range_mask = (self.y_ranges_test == strength_range)
            if np.sum(range_mask) > 0:
                original_within_10 = np.mean(original_errors[range_mask] <= 10) * 100
                meta_within_10 = np.mean(meta_errors[range_mask] <= 10) * 100
                improvement = meta_within_10 - original_within_10
                original_mean_error = np.mean(original_errors[range_mask])
                meta_mean_error = np.mean(meta_errors[range_mask])
                error_reduction = original_mean_error - meta_mean_error
                print(f"Strength Range: {strength_range.replace('_', ' ').title()}")
                print(f"  Original within 10%: {original_within_10:.2f}%")
                print(f"  Meta-learner within 10%: {meta_within_10:.2f}%")
                print(f"  Improvement: {improvement:.2f} percentage points")
                print(f"  Mean error reduction: {error_reduction:.2f}%")
                print("-" * 70)
        return self.meta_metrics, self.meta_preds

    def _create_meta_feature_generator(self):
        """Create a function to generate meta-features for new data."""
        def generate_meta_features(self, X):
            """Generate meta-features for new data samples."""
            deep_preds = self.deep_catboost.predict(X)
            meta_features = [deep_preds]
            for range_name in self.strength_labels:
                if hasattr(self, 'range_models') and range_name in self.range_models:
                    meta_features.append(self.range_models[range_name].predict(X))
            if hasattr(self, 'boundary_models') and self.boundary_models:
                for name, model in self.boundary_models.items():
                    meta_features.append(model.predict(X))
            if hasattr(self, 'age_models') and self.age_models:
                for age_group, model in self.age_models.items():
                    meta_features.append(model.predict(X))
            if hasattr(self, 'very_low_specialized_models') and self.very_low_specialized_models:
                for name, model in self.very_low_specialized_models.items():
                    meta_features.append(model.predict(X))
            if hasattr(self, 'medium_bias_model'):
                bias_corrected_preds = deep_preds.copy()
                medium_mask = (deep_preds >= 40) & (deep_preds < 60)
                if np.any(medium_mask):
                    medium_indices = np.where(medium_mask)[0]
                    X_medium = X.iloc[medium_indices]
                    bias_predictions = self.medium_bias_model.predict(X_medium)
                    for idx, i in enumerate(medium_indices):
                        bias_corrected_preds[i] -= bias_predictions[idx] * 0.7
                meta_features.append(bias_corrected_preds)

            estimated_ranges = pd.cut(deep_preds, bins=self.strength_bins, labels=self.strength_labels)
            range_indicators = pd.get_dummies(estimated_ranges).reindex(columns=self.strength_labels, fill_value=0).values

            # Ensure the number of columns from meta_features_array + range_indicators + X.values matches meta_feature_names
            # If self.meta_feature_names is based on the full X_test.values, X.values should align.
            # However, if X only contains original features and X_test contains engineered, this can cause a mismatch.
            # The safest approach is to ensure X always has all necessary features, either original or engineered,
            # in the correct order for stacking with meta_features_array.

            # It looks like the original code passed X_scaled_df (which has all_features) to generate_meta_features,
            # so X.values here should correspond to self.all_features.

            meta_features_array = np.column_stack(meta_features)

            # The discrepancy likely happens here if X (passed to generate_meta_features) doesn't have the same columns as self.X_test
            # which was used to train the meta-learner. We need to ensure `X` here is `X_scaled_df`.
            # The `predict` method already passes `X_scaled_df`.

            # Adjust range_indicators to have a consistent number of columns if labels change
            expected_range_cols = len(self.strength_labels)
            current_range_cols = range_indicators.shape[1]
            if current_range_cols < expected_range_cols:
                # Add missing columns with zeros
                missing_cols = expected_range_cols - current_range_cols
                range_indicators = np.hstack([range_indicators, np.zeros((range_indicators.shape[0], missing_cols))])
            elif current_range_cols > expected_range_cols:
                # Truncate extra columns (shouldn't happen with reindex but as a safeguard)
                range_indicators = range_indicators[:, :expected_range_cols]


            meta_features_array_final = np.column_stack([meta_features_array, range_indicators, X.values])

            print("✅ Final meta feature shape:", meta_features_array_final.shape)
            if hasattr(self, 'meta_catboost'):
                print("📦 CatBoost expects:", self.meta_catboost.feature_count_)
            else:
                print("📦 Meta-learner not yet trained or found.")


            meta_features_df = pd.DataFrame(meta_features_array_final, columns = self.meta_feature_names)
            return meta_features_df

        self.generate_meta_features = generate_meta_features.__get__(self, self.__class__)


    def _calculate_metrics(self, y_true, y_pred):
        from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        percent_errors = np.abs((y_true - y_pred) / y_true * 100)
        return {
            'r2': r2, 'rmse': rmse, 'mae': mae,
            'max_percent_error': np.max(percent_errors),
            'mean_percent_error': np.mean(percent_errors),
            'median_percent_error': np.median(percent_errors),
            'percent_within_5': np.mean(percent_errors <= 5) * 100,
            'percent_within_10': np.mean(percent_errors <= 10) * 100
        }

    def save_model(self, filepath='models/enhanced_catboost_model.joblib'):
        model_dir = Path('models')
        model_dir.mkdir(exist_ok=True)
        model_data = {
            'deep_catboost': getattr(self, 'deep_catboost', None),
            'range_models': getattr(self, 'range_models', {}),
            'meta_learner': getattr(self, 'meta_learner', None),
            'meta_learner_type': getattr(self, 'meta_learner_type', None),
            'meta_features_scaler': getattr(self, 'meta_features_scaler', None),
            'meta_weights': getattr(self, 'meta_weights', None),
            'meta_catboost': getattr(self, 'meta_catboost', None),
            'meta_mlp': getattr(self, 'meta_mlp', None),
            'meta_feature_names': getattr(self, 'meta_feature_names', None),
            'scaler': self.scaler,
            'original_features': self.original_features,
            'engineered_features': self.engineered_features,
            'all_features': self.all_features,
            'strength_bins': self.strength_bins,
            'strength_labels': self.strength_labels,
            'random_state': self.random_state,
            'catboost_preds': getattr(self, 'catboost_preds', None),
            'meta_preds': getattr(self, 'meta_preds', None),
            'meta_metrics': getattr(self, 'meta_metrics', None)
        }
        joblib.dump(model_data, filepath)
        print(f"Enhanced CatBoost models saved to {filepath}")

    @classmethod
    def load_model(cls, filepath='models/enhanced_catboost_model.joblib'):
        model_data = joblib.load(filepath)
        predictor = cls()
        if 'meta_feature_names' in model_data:
            predictor.meta_feature_names = model_data['meta_feature_names']
        for key, value in model_data.items():
            setattr(predictor, key, value)
        if hasattr(predictor, 'meta_learner'):
            predictor._create_meta_feature_generator()
        return predictor

    def detect_and_correct_outliers(self, X, predictions):
        corrected_predictions = predictions.copy()
        if 'water_cement_ratio' in X.columns and 'abnormal_mix_factor' in X.columns:
            wcr = X['water_cement_ratio']
            abnormal_factor = X['abnormal_mix_factor']
            wcr_array = np.array(wcr)
            abnormal_factor_array = np.array(abnormal_factor)
            wcr_high = wcr_array > np.quantile(wcr_array, 0.95)
            wcr_low = wcr_array < np.quantile(wcr_array, 0.05)
            abnormal_high = abnormal_factor_array > 2.0
            potential_outliers = wcr_high | wcr_low | abnormal_high
            outlier_indices = np.where(potential_outliers)[0]
            if len(outlier_indices) > 0:
                print(f"Detected {len(outlier_indices)} potential outlier predictions")
                for i in outlier_indices:
                    pred_value = predictions[i]
                    if pred_value < 20: strength_range = 'very_low'
                    elif pred_value < 40: strength_range = 'low'
                    elif pred_value < 60: strength_range = 'medium'
                    else: strength_range = 'high'
                    if hasattr(self, 'range_models') and strength_range in self.range_models:
                        range_pred = self.range_models[strength_range].predict(X.iloc[[i]])[0]
                        corrected_predictions[i] = 0.3 * predictions[i] + 0.7 * range_pred
                        print(f"  Outlier at index {i}: Original {predictions[i]:.2f}, Corrected {corrected_predictions[i]:.2f}")
        return corrected_predictions

    def predict(self, X_new):
        if not hasattr(self, 'meta_learner'):
            raise ValueError("Meta-learner has not been trained. Call train_meta_learner first.")

        if isinstance(X_new, pd.DataFrame):
            X_engineered = self.engineer_features(X_new)
        else:
            X_new_df = pd.DataFrame(X_new, columns=self.original_features)
            X_engineered = self.engineer_features(X_new_df)

        # Ensure all_features are present and in the correct order before scaling
        X_aligned = pd.DataFrame(columns=self.all_features)
        for col in self.all_features:
            if col in X_engineered.columns:
                X_aligned[col] = X_engineered[col]
            else:
                X_aligned[col] = 0.0  # Or appropriate default/mean if missing

        X_scaled = pd.DataFrame(
            self.scaler.transform(X_aligned),
            columns=X_aligned.columns
        )

        meta_features = self.generate_meta_features(X_scaled)
        predictions = self.meta_learner.predict(meta_features)
        predictions = self.detect_and_correct_outliers(X_scaled, predictions)

        final_predictions = []
        for i, pred in enumerate(predictions):
            if pred < 20: strength_range = 'very_low'
            elif pred < 40: strength_range = 'low'
            elif pred < 60: strength_range = 'medium'
            else: strength_range = 'high'
            if strength_range == 'very_low':
                if hasattr(self, 'very_low_specialized_models'):
                    if pred < 15 and 'ultra_low' in self.very_low_specialized_models:
                        specialized_pred = self.very_low_specialized_models['ultra_low'].predict([X_scaled.iloc[i]])[0]
                        pred = 0.4 * pred + 0.6 * specialized_pred
                    elif pred >= 15 and pred < 20 and 'mid_low' in self.very_low_specialized_models:
                        specialized_pred = self.very_low_specialized_models['mid_low'].predict([X_scaled.iloc[i]])[0]
                        pred = 0.4 * pred + 0.6 * specialized_pred
            elif strength_range == 'medium':
                if hasattr(self, 'medium_bias_model'):
                    estimated_bias = self.medium_bias_model.predict([X_scaled.iloc[i]])[0]
                    if estimated_bias > 5: pred -= estimated_bias * 0.7
            elif strength_range == 'high': pred *= 1.05
            final_predictions.append(pred)
        return np.array(final_predictions)

In [13]:
def main():
    """Main function to train and evaluate enhanced CatBoost models."""
    # First, ensure CatBoost is installed
    try:
        import subprocess
        import sys

        # Check if running in Colab
        in_colab = 'google.colab' in sys.modules

        if in_colab:
            print("Installing required packages...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "catboost"])
            print("Packages installed successfully.")
    except:
        print("Failed to automatically install packages. Please install manually if needed.")

    print("\nENHANCED CATBOOST MODELS FOR CONCRETE STRENGTH PREDICTION")
    print("="*70)

    # Initialize predictor
    enhanced_predictor = EnhancedCatBoostPredictor(random_state=42)

    # Load and preprocess data
    print("\nLoading and preprocessing data...")
    X_train, X_test, y_train, y_test, y_ranges_train, y_ranges_test = enhanced_predictor.load_and_preprocess('Concrete_Data.xls')

    # Train deep CatBoost model
    deep_metrics, deep_preds = enhanced_predictor.train_deep_catboost()

    # Train range-specific models
    range_models, range_preds = enhanced_predictor.train_range_specific_models()

    # Train boundary models
    boundary_models, boundary_preds = enhanced_predictor.train_boundary_models()

    # Train age-specific models
    age_models, age_preds = enhanced_predictor.train_age_specific_models()

    # Train specialized models for very low strength concrete
    very_low_models, very_low_preds = enhanced_predictor.train_very_low_specialized_models()

    # Train medium range bias correction
    medium_bias_model, medium_bias_preds = enhanced_predictor.train_medium_bias_correction()

    # Train non-linear meta-learner
    meta_metrics, meta_preds = enhanced_predictor.train_meta_learner()

    # Save the final model
    enhanced_predictor.save_model()

    # Detailed Error Analysis
    print("\nDetailed Error Analysis by Strength Range:")
    print("="*70)

    # Function to analyze errors by strength range
    def analyze_predictions(y_true, y_pred, y_ranges, title):
        percent_errors = np.abs((y_true - y_pred) / y_true * 100)

        # Overall metrics
        overall_metrics = {
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2_score(y_true, y_pred),
            'mean_err': np.mean(percent_errors),
            'within_10': np.mean(percent_errors <= 10) * 100,
            'within_5': np.mean(percent_errors <= 5) * 100,
            'count': len(y_true)
        }

        print(f"\n{title} Overall Performance:")
        print(f"  Samples: {overall_metrics['count']}")
        print(f"  R²: {overall_metrics['r2']:.4f}")
        print(f"  RMSE: {overall_metrics['rmse']:.2f} MPa")
        print(f"  Mean % Error: {overall_metrics['mean_err']:.2f}%")
        print(f"  Within 5%: {overall_metrics['within_5']:.2f}%")
        print(f"  Within 10%: {overall_metrics['within_10']:.2f}%")

        # Count high-error samples
        high_error_count = np.sum(percent_errors > 10)
        print(f"  Samples with >10% error: {high_error_count} ({high_error_count/len(y_true)*100:.2f}%)")

        # Metrics by strength range
        print(f"\n{title} Performance by Strength Range:")
        print("-" * 70)
        print(f"{'Strength Range':<20} {'Count':<8} {'RMSE':<8} {'R²':<8} {'Mean %':<8} {'Within 10%':<10} {'Within 5%':<10}")
        print("-" * 70)

        for strength_range in enhanced_predictor.strength_labels:
            mask = (y_ranges == strength_range)
            if np.sum(mask) > 0:
                y_range = y_true[mask]
                y_pred_range = y_pred[mask]
                percent_errors_range = percent_errors[mask]

                rmse = np.sqrt(mean_squared_error(y_range, y_pred_range))
                r2 = r2_score(y_range, y_pred_range)
                mean_percent = np.mean(percent_errors_range)
                within_10 = np.mean(percent_errors_range <= 10) * 100
                within_5 = np.mean(percent_errors_range <= 5) * 100

                print(f"{strength_range.replace('_', ' ').title():<20} {np.sum(mask):<8d} {rmse:<8.2f} {r2:<8.4f} {mean_percent:<8.2f} {within_10:<10.2f} {within_5:<10.2f}")

        # Return high error samples for further analysis
        return y_true[percent_errors > 10], y_pred[percent_errors > 10], percent_errors[percent_errors > 10]

    # Analyze deep CatBoost predictions
    if deep_preds is not None:
        print("\n" + "="*70)
        print("DEEP CATBOOST MODEL ANALYSIS")
        high_err_true, high_err_pred, high_errs = analyze_predictions(
            y_test, deep_preds, y_ranges_test, "Deep CatBoost"
        )

    # Analyze meta-learner predictions
    if meta_preds is not None:
        print("\n" + "="*70)
        print("META-LEARNER MODEL ANALYSIS")
        high_err_true_meta, high_err_pred_meta, high_errs_meta = analyze_predictions(
            y_test, meta_preds, y_ranges_test, "Meta-Learner"
        )

        # Compare high error samples
        if deep_preds is not None:
            deep_high_error_count = np.sum(np.abs((y_test - deep_preds) / y_test * 100) > 10)
            meta_high_error_count = np.sum(np.abs((y_test - meta_preds) / y_test * 100) > 10)

            print("\nHigh Error Sample Comparison:")
            print(f"  Deep CatBoost high-error samples: {deep_high_error_count}")
            print(f"  Meta-learner high-error samples: {meta_high_error_count}")
            print(f"  Improvement: {deep_high_error_count - meta_high_error_count} samples")

            # Find improved and worsened samples
            deep_errors = np.abs((y_test - deep_preds) / y_test * 100)
            meta_errors = np.abs((y_test - meta_preds) / y_test * 100)

            improved_mask = (deep_errors > 10) & (meta_errors <= 10)
            worsened_mask = (deep_errors <= 10) & (meta_errors > 10)

            print(f"  Samples improved (from >10% to ≤10%): {np.sum(improved_mask)}")
            print(f"  Samples worsened (from ≤10% to >10%): {np.sum(worsened_mask)}")

    def analyze_high_error_samples(enhanced_predictor, X_test, y_test, y_ranges_test, meta_preds):
        print("\n" + "="*80)
        print("HIGH ERROR SAMPLES ANALYSIS")
        print("="*80)

        # 1. Identify high error samples - convert to numpy arrays to avoid index alignment issues
        y_test_np = np.array(y_test)
        meta_preds_np = np.array(meta_preds)
        percent_errors_np = np.abs((y_test_np - meta_preds_np) / y_test_np * 100)
        high_error_mask_np = percent_errors_np > 10

        # Get the indices of high error samples
        high_error_indices = np.where(high_error_mask_np)[0]

        # Use iloc to select rows by position, avoiding index alignment issues
        high_error_samples = X_test.iloc[high_error_indices].copy().reset_index(drop=True)
        high_error_targets = y_test.iloc[high_error_indices].copy().reset_index(drop=True)
        high_error_preds = meta_preds_np[high_error_indices]
        high_error_percent = percent_errors_np[high_error_indices]

        # Get normal samples (not high error)
        normal_indices = np.where(~high_error_mask_np)[0]
        normal_samples = X_test.iloc[normal_indices].copy().reset_index(drop=True)
        normal_targets = y_test.iloc[normal_indices].copy().reset_index(drop=True)

        # Basic statistics
        total_samples = len(y_test)
        high_error_count = len(high_error_indices)
        high_error_rate = high_error_count / total_samples * 100

        print(f"Total test samples: {total_samples}")
        print(f"Samples with >10% error: {high_error_count} ({high_error_rate:.1f}%)")
        print(f"Mean percent error in high-error samples: {np.mean(high_error_percent):.2f}%")
        print(f"Max percent error: {np.max(high_error_percent):.2f}%")

        # 2. Create analysis DataFrame
        error_analysis_df = pd.DataFrame({
            'Actual_Strength': high_error_targets,
            'Predicted_Strength': high_error_preds,
            'Percent_Error': high_error_percent,
            'Strength_Range': [y_ranges_test.iloc[i] for i in high_error_indices],
            'Error_Direction': np.sign(high_error_preds - np.array(high_error_targets))
        })

        # Add original and engineered features
        for col in X_test.columns:
            error_analysis_df[col] = high_error_samples[col].values

        # 3. Analyze distribution by strength range
        print("\n" + "-"*80)
        print("STRENGTH RANGE ANALYSIS")
        print("-"*80)

        # Get y_ranges_test as numpy array to avoid index alignment issues
        y_ranges_test_np = np.array(y_ranges_test)

        # Count errors by strength range
        high_error_ranges = y_ranges_test_np[high_error_indices]
        range_error_counts = pd.Series(high_error_ranges).value_counts()

        # Get total counts by range
        all_ranges = pd.Series(y_ranges_test_np).value_counts()

        # Calculate percentage of errors in each range
        error_percentage_by_range = pd.DataFrame({
            'Total_Samples': all_ranges,
            'Error_Samples': range_error_counts.reindex(index=all_ranges.index, fill_value=0),
            'Error_Rate': range_error_counts.reindex(index=all_ranges.index, fill_value=0) / all_ranges * 100
        }).sort_values('Error_Rate', ascending=False)

        print("Error rates by strength range:")
        print(error_percentage_by_range)

        # 4. Error direction by range
        print("\nError direction by strength range (negative = under-prediction):")
        direction_by_range = error_analysis_df.groupby('Strength_Range')['Error_Direction'].mean()
        print(direction_by_range)

        # 5. Material composition analysis
        print("\n" + "-"*80)
        print("MATERIAL COMPOSITION ANALYSIS")
        print("-"*80)

        # Get original features (not engineered)
        original_features = enhanced_predictor.original_features

        # Calculate average composition for high-error vs normal samples
        high_error_composition = high_error_samples[original_features].mean()
        normal_composition = normal_samples[original_features].mean()

        composition_comparison = pd.DataFrame({
            'High_Error_Avg': high_error_composition,
            'Normal_Samples_Avg': normal_composition,
            'Ratio': high_error_composition / normal_composition,
            'Difference': high_error_composition - normal_composition
        })

        print("Composition comparison (sorted by largest difference from normal samples):")
        print(composition_comparison.sort_values('Difference', key=abs, ascending=False))

        # Statistical significance of differences
        print("\nStatistically significant differences (t-test, p < 0.05):")
        for feature in original_features:
            t, p = ttest_ind(high_error_samples[feature], normal_samples[feature], equal_var=False)
            if p < 0.05:
                print(f"  {feature}: p-value = {p:.4f}, t-statistic = {t:.2f}")

        # 6. Correlation with error size
        print("\n" + "-"*80)
        print("ERROR CORRELATION ANALYSIS")
        print("-"*80)

        # Check all features, including engineered ones
        all_features = X_test.columns.tolist()

        # Create dataframe with features and error for correlation
        X_with_errors = X_test.copy()
        X_with_errors['percent_error'] = percent_errors_np

        # Correlation between features and error magnitude
        error_correlations = {}
        for col in all_features:
            corr = np.corrcoef(X_with_errors[col], X_with_errors['percent_error'])[0, 1]
            error_correlations[col] = corr

        error_corr_df = pd.DataFrame({
            'Feature': list(error_correlations.keys()),
            'Error_Correlation': list(error_correlations.values())
        }).sort_values('Error_Correlation', key=abs, ascending=False)

        print("Top 10 features correlated with error magnitude:")
        print(error_corr_df.head(10))

        # 7. Boundary case analysis
        print("\n" + "-"*80)
        print("BOUNDARY CASE ANALYSIS")
        print("-"*80)

        # Check if errors occur at boundaries between strength ranges
        error_analysis_df['Actual_Near_Boundary'] = False

        for i in range(1, len(enhanced_predictor.strength_bins)-1):
            boundary = enhanced_predictor.strength_bins[i]
            window = 2.0  # MPa window around boundary

            # Direct array access without boolean indexing
            for idx, target in enumerate(high_error_targets):
                if boundary - window <= target <= boundary + window:
                    error_analysis_df.at[idx, 'Actual_Near_Boundary'] = True

        boundary_percentage = error_analysis_df['Actual_Near_Boundary'].mean() * 100
        print(f"Percentage of high-error samples near strength range boundaries: {boundary_percentage:.1f}%")

        # 8. Age analysis
        print("\n" + "-"*80)
        print("AGE ANALYSIS")
        print("-"*80)

        # Define age groups
        age_bins = [0, 7, 28, 90, float('inf')]
        age_labels = ['Very_Young', 'Young', 'Standard', 'Old']

        age_col = 'Age (day)'

        # Create age groups for all samples
        all_age_groups = pd.cut(X_test[age_col], bins=age_bins, labels=age_labels)
        error_age_groups = pd.cut(high_error_samples[age_col], bins=age_bins, labels=age_labels)

        # Count samples in each age group
        age_group_counts = error_age_groups.value_counts()
        total_by_age = all_age_groups.value_counts()

        # Ensure all categories appear in both Series
        age_group_counts = age_group_counts.reindex(index=total_by_age.index, fill_value=0)

        age_error_analysis = pd.DataFrame({
            'Total_Samples': total_by_age,
            'Error_Samples': age_group_counts,
            'Error_Rate': age_group_counts / total_by_age * 100
        }).sort_values('Error_Rate', ascending=False)

        print("Error rates by concrete age:")
        print(age_error_analysis)

        # 9. Create visualizations
        print("\n" + "-"*80)
        print("CREATING VISUALIZATIONS")
        print("-"*80)

        # Set up the plotting style
        sns.set(style='whitegrid')
        plt.rcParams['figure.figsize'] = (12, 8)

        # 9.1 PCA visualization to detect outliers
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_test)

        plt.figure(figsize=(12, 10))
        plt.scatter(X_pca[normal_indices, 0], X_pca[normal_indices, 1],
                    c='blue', alpha=0.5, label='Normal samples')
        plt.scatter(X_pca[high_error_indices, 0], X_pca[high_error_indices, 1],
                    c='red', s=100, alpha=0.7, label='High error samples')
        plt.title('PCA of Test Samples')
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.legend()
        plt.savefig('high_error_pca.png')
        plt.close()
        print("Saved PCA visualization to 'high_error_pca.png'")

        # 9.2 Error vs Actual Strength
        plt.figure(figsize=(12, 8))
        scatter = plt.scatter(y_test_np, percent_errors_np, alpha=0.6, c=y_test_np, cmap='viridis')
        plt.axhline(y=10, color='red', linestyle='--', label='10% Error Threshold')
        for i, range_bound in enumerate(enhanced_predictor.strength_bins[1:-1], 1):
            plt.axvline(x=range_bound, color='gray', linestyle=':',
                        label=f'Range Boundary {range_bound} MPa' if i==1 else None)

        plt.xlabel('Actual Strength (MPa)')
        plt.ylabel('Percent Error (%)')
        plt.title('Percent Error vs. Actual Strength')
        plt.colorbar(label='Strength (MPa)')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.savefig('error_vs_strength.png')
        plt.close()
        print("Saved Error vs Strength visualization to 'error_vs_strength.png'")

        # 9.3 Top 4 features with highest error correlation
        top_error_features = error_corr_df.head(4)['Feature'].values

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.flatten()

        for i, feature in enumerate(top_error_features):
            ax = axes[i]
            scatter = ax.scatter(X_test[feature], percent_errors_np,
                      alpha=0.6, c=y_test_np, cmap='viridis')
            ax.axhline(y=10, color='red', linestyle='--', label='10% Error Threshold')
            ax.set_xlabel(feature)
            ax.set_ylabel('Percent Error (%)' if i % 2 == 0 else '')
            ax.set_title(f'Error vs {feature}\nCorrelation: {error_correlations[feature]:.3f}')
            ax.grid(True, alpha=0.3)

        plt.colorbar(scatter, ax=axes, label='Strength (MPa)')
        plt.tight_layout()
        plt.savefig('error_vs_top_features.png')
        plt.close()
        print("Saved Error vs Top Features visualization to 'error_vs_top_features.png'")

        # 9.4 Key Ratio Histograms
        critical_ratios = ['water_cement_ratio', 'water_cementitious_ratio',
                          'agg_cement_ratio', 'supplementary_fraction',
                          'paste_volume', 'log_age']

        fig, axes = plt.subplots(3, 2, figsize=(15, 15))
        axes = axes.flatten()

        for i, ratio in enumerate(critical_ratios):
            if ratio in X_test.columns:
                ax = axes[i]
                sns.histplot(normal_samples[ratio], ax=ax, color='blue',
                            alpha=0.5, label='Normal samples', kde=True)
                sns.histplot(high_error_samples[ratio], ax=ax, color='red',
                            alpha=0.5, label='High error samples', kde=True)
                ax.set_title(f'Distribution of {ratio}')
                ax.legend()

        plt.tight_layout()
        plt.savefig('ratio_distributions.png')
        plt.close()
        print("Saved Ratio Distributions to 'ratio_distributions.png'")

        # 10. Summary of key findings
        print("\n" + "-"*80)
        print("SUMMARY OF KEY FINDINGS")
        print("-"*80)

        # Most problematic strength range
        if not error_percentage_by_range.empty:
            worst_range = error_percentage_by_range.index[0]
            worst_range_rate = error_percentage_by_range.iloc[0]['Error_Rate']

            print("1. Most problematic strength range:")
            print(f"   {worst_range} with {worst_range_rate:.1f}% error rate")
        else:
            print("1. No problematic strength range identified (no high error samples)")

        # Most correlated feature with errors
        if not error_corr_df.empty:
            top_error_feature = error_corr_df.iloc[0]['Feature']
            top_error_correlation = error_corr_df.iloc[0]['Error_Correlation']

            print("\n2. Feature most correlated with prediction errors:")
            print(f"   {top_error_feature} (correlation: {top_error_correlation:.3f})")
        else:
            print("\n2. No features correlated with prediction errors")

        # Significant material differences
        significant_diffs = []
        for feature in original_features:
            t, p = ttest_ind(high_error_samples[feature], normal_samples[feature], equal_var=False)
            if p < 0.05:
                significant_diffs.append(feature)

        if significant_diffs:
            print("\n3. Significant material composition differences:")
            for feature in significant_diffs:
                high_avg = high_error_samples[feature].mean()
                normal_avg = normal_samples[feature].mean()
                diff_pct = (high_avg - normal_avg) / normal_avg * 100
                print(f"   {feature}: {diff_pct:+.1f}% different from normal samples")
        else:
            print("\n3. No significant material composition differences")

        print("\n4. Boundary effect:")
        if boundary_percentage > 30:
            print(f"   Strong boundary effect: {boundary_percentage:.1f}% of errors near range boundaries")
        else:
            print(f"   Limited boundary effect: {boundary_percentage:.1f}% of errors near range boundaries")

        print("\n5. Age effect:")
        if not age_error_analysis.empty:
            problematic_age = age_error_analysis.index[0]
            problematic_age_rate = age_error_analysis.iloc[0]['Error_Rate']
            print(f"   Most problematic age group: {problematic_age} with {problematic_age_rate:.1f}% error rate")
        else:
            print("   No age effect identified")

        # 11. Save detailed error analysis to file
        error_analysis_df.to_csv('high_error_samples_analysis.csv', index=False)
        print("\nDetailed error analysis saved to 'high_error_samples_analysis.csv'")

        # 12. Return high error samples for further investigation
        return error_analysis_df, error_corr_df, high_error_samples, high_error_targets, high_error_preds, high_error_percent, normal_samples, normal_targets


    # Save detailed results
    results_df = pd.DataFrame({
        'Actual_Strength': y_test,
        'Deep_CatBoost_Prediction': deep_preds if deep_preds is not None else None,
        'Meta_Learner_Prediction': meta_preds if meta_preds is not None else None,
        'Strength_Range': y_ranges_test,
        'Deep_CatBoost_Error_Pct': np.abs((y_test - deep_preds) / y_test * 100) if deep_preds is not None else None,
        'Meta_Learner_Error_Pct': np.abs((y_test - meta_preds) / y_test * 100) if meta_preds is not None else None
    })

    # Add improvement column
    if deep_preds is not None and meta_preds is not None:
        results_df['Error_Improvement'] = results_df['Deep_CatBoost_Error_Pct'] - results_df['Meta_Learner_Error_Pct']

    # Perform detailed analysis on the high-error samples
    print("\n" + "="*70)
    print("DETAILED HIGH-ERROR SAMPLES ANALYSIS")

    error_df, corr_df, high_error_samples, high_error_targets, high_error_preds, high_error_percent, normal_samples, normal_targets = analyze_high_error_samples(
        enhanced_predictor=enhanced_predictor,
        X_test=X_test,
        y_test=y_test,
        y_ranges_test=y_ranges_test,
        meta_preds=meta_preds
    )

    # Add information to results dataframe for further inspection
    results_df['Is_High_Error'] = np.abs((y_test - meta_preds) / y_test * 100) > 10

    # Save results
    results_df.to_csv('enhanced_catboost_results.csv', index=False)
    print("\nDetailed results saved to 'enhanced_catboost_results.csv'")

    print("\nEnhanced CatBoost training and evaluation complete!")

if __name__ == "__main__":
    main()

Installing required packages...


INFO:__main__:Data loaded successfully
INFO:__main__:Created 24 new engineered features


Packages installed successfully.

ENHANCED CATBOOST MODELS FOR CONCRETE STRENGTH PREDICTION

Loading and preprocessing data...
Data split: (824, 32) training, (206, 32) testing

Strength range distribution in test set:
  Very Low: 39 samples (18.9%)
  Low: 91 samples (44.2%)
  Medium: 57 samples (27.7%)
  High: 19 samples (9.2%)

Training deep CatBoost model...
0:	learn: 16.3773542	test: 16.8433682	best: 16.8433682 (0)	total: 24.7ms	remaining: 49.4s
100:	learn: 6.1136361	test: 6.8700188	best: 6.8700188 (100)	total: 1.81s	remaining: 34.1s
200:	learn: 4.2346431	test: 5.4830746	best: 5.4830746 (200)	total: 4.03s	remaining: 36.1s
300:	learn: 3.5294387	test: 5.0404466	best: 5.0404466 (300)	total: 6.84s	remaining: 38.6s
400:	learn: 3.0696553	test: 4.7563153	best: 4.7563153 (400)	total: 8.62s	remaining: 34.4s
500:	learn: 2.7190146	test: 4.5948980	best: 4.5948980 (500)	total: 10.5s	remaining: 31.3s
600:	learn: 2.4682995	test: 4.4794460	best: 4.4794460 (600)	total: 12.3s	remaining: 28.5s
700:	l

**TEST
PREDICTION**

In [14]:
def predict_concrete_strength():
    """
    Interactive function to predict concrete strength using the
    trained Enhanced CatBoost model.
    """
    print("\n==== Concrete Strength Prediction Tool ====\n")

    # Try to load the saved model
    model_path = Path('models/enhanced_catboost_model.joblib')

    if not model_path.exists():
        print(f"Error: Model file not found at {model_path}")
        print("Please ensure you've trained and saved the model first.")
        return

    try:
        # Load the model
        print("Loading model...", end='')
        model_data = joblib.load(model_path)
        predictor = EnhancedCatBoostPredictor()

        for key, value in model_data.items():
            setattr(predictor, key, value)

        # Fix the _create_meta_feature_generator method to handle the error
        def fixed_generate_meta_features(self, X):
            """Fixed version of generate_meta_features that handles the error."""
            # Generate predictions from deep CatBoost
            deep_preds = self.deep_catboost.predict(X)

            # Initialize the meta_features list
            meta_features = [deep_preds]

            # Generate predictions from range-specific models
            for range_name in self.strength_labels:
                if hasattr(self, 'range_models') and range_name in self.range_models:
                    range_preds = self.range_models[range_name].predict(X)
                    meta_features.append(range_preds)

            # Generate predictions from boundary models if available
            if hasattr(self, 'boundary_models') and self.boundary_models:
                for name, model in self.boundary_models.items():
                    boundary_preds = model.predict(X)
                    meta_features.append(boundary_preds)

            # Generate predictions from age-specific models if available
            if hasattr(self, 'age_models') and self.age_models:
                for age_group, model in self.age_models.items():
                    age_preds = model.predict(X)
                    meta_features.append(age_preds)

            # Generate predictions from very low specialized models if available
            if hasattr(self, 'very_low_specialized_models') and self.very_low_specialized_models:
                for name, model in self.very_low_specialized_models.items():
                    specialized_preds = model.predict(X)
                    meta_features.append(specialized_preds)

            # Generate predictions with medium bias correction if available
            if hasattr(self, 'medium_bias_model'):
                # Create bias-corrected predictions
                bias_corrected_preds = deep_preds.copy()

                # Estimate which samples are in medium range
                medium_mask = (deep_preds >= 40) & (deep_preds < 60)

                # For medium range samples, apply bias correction
                if np.any(medium_mask):
                    # Get medium range indices
                    medium_indices = np.where(medium_mask)[0]

                    # Get medium range samples
                    X_medium = X.iloc[medium_indices]

                    # Get bias predictions for medium range samples
                    bias_predictions = self.medium_bias_model.predict(X_medium)

                    # Apply bias correction (70% of predicted bias)
                    for idx, i in enumerate(medium_indices):
                        bias_corrected_preds[i] -= bias_predictions[idx] * 0.7

                meta_features.append(bias_corrected_preds)

            # Determine strength range indicators
            # Since we don't know the true range, we'll estimate based on deep model
            estimated_ranges = pd.cut(
                deep_preds,
                bins=self.strength_bins,
                labels=self.strength_labels
            )

            # Convert to dummy variables (one-hot encoding)
            range_indicators = pd.get_dummies(pd.Series(estimated_ranges)).values

            # Stack all meta-features
            meta_features_array = np.column_stack(meta_features)

            # Add range indicators and original features
            meta_features_array = np.column_stack([
                meta_features_array,
                range_indicators,
                X.values  # Add original features
            ])

            # Scale if needed
            if hasattr(self, 'meta_learner_type') and self.meta_learner_type == 'mlp':
                meta_features_array = self.meta_features_scaler.transform(meta_features_array)

            return meta_features_array

        # Replace the method
        predictor.generate_meta_features = lambda X: fixed_generate_meta_features(predictor, X)

        print(" Done!")

        # Get user inputs
        print("\nPlease enter the concrete mix parameters:")

        # Create a dictionary to store inputs
        inputs = {}

        # Define the required inputs and their descriptions
        input_fields = [
            ('Cement (component 1)(kg in a m^3 mixture)', 'Cement content (kg/m³)'),
            ('Blast Furnace Slag (component 2)(kg in a m^3 mixture)', 'Blast Furnace Slag content (kg/m³)'),
            ('Fly Ash (component 3)(kg in a m^3 mixture)', 'Fly Ash content (kg/m³)'),
            ('Water  (component 4)(kg in a m^3 mixture)', 'Water content (kg/m³)'),
            ('Superplasticizer (component 5)(kg in a m^3 mixture)', 'Superplasticizer content (kg/m³)'),
            ('Coarse Aggregate  (component 6)(kg in a m^3 mixture)', 'Coarse Aggregate content (kg/m³)'),
            ('Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Fine Aggregate content (kg/m³)'),
            ('Age (day)', 'Concrete age (days)')
        ]

        # Collect all inputs
        for field, description in input_fields:
            while True:
                try:
                    value = float(input(f"{description}: "))
                    inputs[field] = value
                    break
                except ValueError:
                    print("  Please enter a valid number.")

        # Create a DataFrame with the inputs
        input_df = pd.DataFrame([inputs])

        # Make prediction
        print("\nCalculating concrete strength prediction...")

        # Engineer features with the loaded model
        X_engineered = predictor.engineer_features(input_df)

        # Align columns before scaling
        X_engineered = X_engineered[predictor.all_features]

        # Scale features
        X_scaled = pd.DataFrame(
            predictor.scaler.transform(X_engineered),
            columns=X_engineered.columns
        )

        # Get prediction from model
        try:
            # Try direct prediction first
            if hasattr(predictor, 'meta_learner'):
                # Use meta-learner prediction
                meta_features = predictor.generate_meta_features(X_scaled)
                prediction = predictor.meta_learner.predict(meta_features)[0]
            else:
                # Fallback to deep CatBoost model
                prediction = predictor.deep_catboost.predict(X_scaled)[0]

            # Apply range-specific corrections
            if prediction < 20:
                strength_range = "Very Low Strength"
                # Check for specialized very low models
                if hasattr(predictor, 'very_low_specialized_models'):
                    if prediction < 15 and 'ultra_low' in predictor.very_low_specialized_models:
                        specialized_pred = predictor.very_low_specialized_models['ultra_low'].predict(X_scaled)[0]
                        prediction = 0.4 * prediction + 0.6 * specialized_pred
                    elif 'mid_low' in predictor.very_low_specialized_models:
                        specialized_pred = predictor.very_low_specialized_models['mid_low'].predict(X_scaled)[0]
                        prediction = 0.4 * prediction + 0.6 * specialized_pred
            elif prediction < 40:
                strength_range = "Low Strength"
            elif prediction < 60:
                strength_range = "Medium Strength"
                # Apply bias correction for medium range
                if hasattr(predictor, 'medium_bias_model'):
                    estimated_bias = predictor.medium_bias_model.predict(X_scaled)[0]
                    if estimated_bias > 2:
                        prediction -= estimated_bias * 0.7
            else:
                strength_range = "High Strength"
                # Boost high strength predictions slightly
                prediction *= 1.02
        except Exception as e:
            print(f"Error in prediction: {str(e)}")
            print("Falling back to deep CatBoost model...")
            prediction = predictor.deep_catboost.predict(X_scaled)[0]

            # Simple determination of strength range
            if prediction < 20:
                strength_range = "Very Low Strength"
            elif prediction < 40:
                strength_range = "Low Strength"
            elif prediction < 60:
                strength_range = "Medium Strength"
            else:
                strength_range = "High Strength"

        # Display results
        print("\n==== Prediction Results ====")
        print(f"Predicted Concrete Strength: {prediction:.2f} MPa")
        print(f"Strength Classification: {strength_range}")

        # Calculate and display key engineering ratios
        w_c_ratio = inputs['Water  (component 4)(kg in a m^3 mixture)'] / inputs['Cement (component 1)(kg in a m^3 mixture)']
        total_cementitious = inputs['Cement (component 1)(kg in a m^3 mixture)'] + \
                            inputs['Blast Furnace Slag (component 2)(kg in a m^3 mixture)'] + \
                            inputs['Fly Ash (component 3)(kg in a m^3 mixture)']
        w_cm_ratio = inputs['Water  (component 4)(kg in a m^3 mixture)'] / total_cementitious

        print("\nKey Engineering Ratios:")
        print(f"Water-Cement Ratio: {w_c_ratio:.3f}")
        print(f"Water-Cementitious Materials Ratio: {w_cm_ratio:.3f}")
        print(f"Total Cementitious Materials: {total_cementitious:.1f} kg/m³")

        # Provide feedback based on the prediction
        if w_cm_ratio > 0.5 and prediction > 40:
            print("- Note: The water-cementitious ratio is relatively high for the predicted strength.")
        if total_cementitious < 300 and prediction > 50:
            print("- Note: The cementitious content is relatively low for the predicted strength.")
        if inputs['Age (day)'] < 28:
            print(f"- Estimated 28-day strength would be higher than the current prediction.")

    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the prediction tool if this script is executed directly
if __name__ == "__main__":
    predict_concrete_strength()


==== Concrete Strength Prediction Tool ====

Loading model... Done!

Please enter the concrete mix parameters:
Cement content (kg/m³): 500
Blast Furnace Slag content (kg/m³): 0
Fly Ash content (kg/m³): 0
Water content (kg/m³): 160
Superplasticizer content (kg/m³): 2
Coarse Aggregate content (kg/m³): 1000
Fine Aggregate content (kg/m³): 670
Concrete age (days): 25

Calculating concrete strength prediction...

==== Prediction Results ====
Predicted Concrete Strength: 37.67 MPa
Strength Classification: Low Strength

Key Engineering Ratios:
Water-Cement Ratio: 0.320
Water-Cementitious Materials Ratio: 0.320
Total Cementitious Materials: 500.0 kg/m³
- Estimated 28-day strength would be higher than the current prediction.


In [20]:
def print_meta_learner_equation(predictor, input_data):
    """
    Prints the actual equation showing each model's contribution to the final prediction
    for a specific input case.

    Args:
        predictor: Trained EnhancedCatBoostPredictor object
        input_data: DataFrame with a single row containing the input concrete mix
    """
    # Process the input data
    X_engineered = predictor.engineer_features(input_data)
    X_scaled = pd.DataFrame(
        predictor.scaler.transform(X_engineered),
        columns=predictor.all_features # Use all_features as columns
    )


    # Get predictions from individual models
    print("\n=== META-LEARNER EQUATION BREAKDOWN ===\n")

    # Dictionary to store all predictions
    all_predictions = {}

    # 1. Deep CatBoost prediction
    deep_pred = predictor.deep_catboost.predict(X_scaled)[0]
    all_predictions["Deep CatBoost"] = deep_pred
    print(f"Deep CatBoost prediction: {deep_pred:.2f} MPa")

    # 2. Range-specific model predictions
    print("\nRange-specific models:")
    for range_name in predictor.strength_labels:
        if hasattr(predictor, 'range_models') and range_name in predictor.range_models:
            pred = predictor.range_models[range_name].predict(X_scaled)[0]
            all_predictions[f"Range-{range_name}"] = pred
            print(f"  {range_name.replace('_', ' ').title()} range model: {pred:.2f} MPa")

    # 3. Boundary model predictions
    if hasattr(predictor, 'boundary_models') and predictor.boundary_models:
        print("\nBoundary region models:")
        for name in predictor.boundary_models:
            pred = predictor.boundary_models[name].predict(X_scaled)[0]
            all_predictions[f"Boundary-{name}"] = pred
            print(f"  {name.replace('_', ' ').title()} model: {pred:.2f} MPa")

    # 4. Age-specific model predictions
    if hasattr(predictor, 'age_models') and predictor.age_models:
        print("\nAge-specific models:")
        for age_group in predictor.age_models:
            pred = predictor.age_models[age_group].predict(X_scaled)[0]
            all_predictions[f"Age-{age_group}"] = pred
            print(f"  {age_group.replace('_', ' ').title()} age model: {pred:.2f} MPa")

    # 5. Specialized model predictions for very low strength
    if hasattr(predictor, 'very_low_specialized_models') and predictor.very_low_specialized_models:
        print("\nVery low specialized models:")
        for name in predictor.very_low_specialized_models:
            pred = predictor.very_low_specialized_models[name].predict(X_scaled)[0]
            all_predictions[f"VeryLow-{name}"] = pred
            print(f"  {name.replace('_', ' ').title()} model: {pred:.2f} MPa")

    # 6. Medium bias correction
    if hasattr(predictor, 'medium_bias_model'):
        bias = predictor.medium_bias_model.predict(X_scaled)[0]
        bias_corrected = deep_pred - (bias * 0.7)
        all_predictions["MediumBias-corrected"] = bias_corrected
        print(f"\nMedium bias correction:")
        print(f"  Estimated bias: {bias:.2f} MPa")
        print(f"  Bias-corrected prediction: {bias_corrected:.2f} MPa")

    # Generate meta-features
    meta_features = predictor.generate_meta_features(X_scaled)

    # Get the meta-learner prediction
    meta_pred = deep_pred # Default to deep_pred

    if hasattr(predictor, 'meta_learner') and predictor.meta_learner is not None:
        meta_pred = predictor.meta_learner.predict(meta_features)[0]
        print(f"\nMeta-learner raw prediction: {meta_pred:.2f} MPa")
    elif hasattr(predictor, 'meta_catboost') and hasattr(predictor, 'meta_mlp') and \
         predictor.meta_catboost is not None and predictor.meta_mlp is not None and hasattr(predictor, 'meta_weights'):
        # For ensemble meta-learner
        catboost_pred = predictor.meta_catboost.predict(meta_features)[0]
        mlp_pred = predictor.meta_mlp.predict(meta_features)[0]

        weights = predictor.meta_weights
        # Ensure weights is a tuple with at least two elements
        if isinstance(weights, tuple) and len(weights) >= 2:
            weighted_pred = (weights[0] * catboost_pred) + (weights[1] * mlp_pred)
            print(f"\nMeta-learner components:")
            print(f"  CatBoost meta-learner: {catboost_pred:.2f} MPa (weight: {weights[0]:.2f})")
            print(f"  MLP meta-learner: {mlp_pred:.2f} MPa (weight: {weights[1]:.2f})")
            print(f"  Weighted ensemble: {weighted_pred:.2f} MPa")
            meta_pred = weighted_pred
        else:
             print("\nMeta-learner weights not in expected format (tuple). Using deep CatBoost prediction.")
             meta_pred = deep_pred
    else:
        print("\nNo working meta-learner found, using deep CatBoost prediction")


    # Determine final prediction with range-specific corrections
    if meta_pred < 20:
        strength_range = "very_low"
        # Apply specialized corrections for very low
        if hasattr(predictor, 'very_low_specialized_models'):
            if meta_pred < 15 and 'ultra_low' in predictor.very_low_specialized_models:
                specialized_pred = all_predictions.get("VeryLow-ultra_low", 0)
                corrected_pred = 0.4 * meta_pred + 0.6 * specialized_pred
                print(f"\nVery low range correction:")
                print(f"  Ultra-low model blend: (0.4 × {meta_pred:.2f}) + (0.6 × {specialized_pred:.2f}) = {corrected_pred:.2f} MPa")
                meta_pred = corrected_pred
            elif meta_pred >= 15 and 'mid_low' in predictor.very_low_specialized_models:
                specialized_pred = all_predictions.get("VeryLow-mid_low", 0)
                corrected_pred = 0.4 * meta_pred + 0.6 * specialized_pred
                print(f"\nVery low range correction:")
                print(f"  Mid-low model blend: (0.4 × {meta_pred:.2f}) + (0.6 × {specialized_pred:.2f}) = {corrected_pred:.2f} MPa")
                meta_pred = corrected_pred
    elif meta_pred < 40:
        strength_range = "low"
    elif meta_pred < 60:
        strength_range = "medium"
        # Apply medium bias correction
        if hasattr(predictor, 'medium_bias_model'):
            bias = predictor.medium_bias_model.predict(X_scaled)[0]
            if bias > 2:
                corrected_pred = meta_pred - (bias * 0.7)
                print(f"\nMedium range correction:")
                print(f"  Bias correction: {meta_pred:.2f} - (0.7 × {bias:.2f}) = {corrected_pred:.2f} MPa")
                meta_pred = corrected_pred
    else:
        strength_range = "high"
        # Boost high strength predictions
        corrected_pred = meta_pred * 1.02
        print(f"\nHigh range correction:")
        print(f"  High strength boost: {meta_pred:.2f} × 1.02 = {corrected_pred:.2f} MPa")
        meta_pred = corrected_pred

    # Print final meta-learner equation
    print("\n=== FINAL META-LEARNER EQUATION ===\n")
    print(f"For this concrete mix, the model predicts: {meta_pred:.2f} MPa ({strength_range.replace('_', ' ').title()} Strength)")

    # Try to create an approximation of the meta-learner equation
    print("\nApproximating the ensemble equation:")

    # Check if we have access to meta-learner weights
    if hasattr(predictor, 'meta_weights') and isinstance(predictor.meta_weights, tuple) and len(predictor.meta_weights) >= 2:
        # Use saved weights
        weights_dict = {'CatBoost': predictor.meta_weights[0], 'MLP': predictor.meta_weights[1]}
        weight_info = f"Using saved meta-weights: CatBoost={weights_dict['CatBoost']:.2f}, MLP={weights_dict['MLP']:.2f}"
        weights_source = 'saved'
    else:
        # Estimate weights based on strength range
        weights_dict = {}
        total_weight = 0

        # Assign weights based on strength range relevance
        for key in all_predictions:
            # Use a more robust check for range names
            key_lower = key.lower()
            if strength_range.replace('_', ' ') in key_lower:
                weights_dict[key] = 0.20  # Higher weight for relevant range
            elif "deep catboost" in key_lower:
                weights_dict[key] = 0.30  # Main model
            elif "boundary" in key_lower and (strength_range.replace('_', ' ') in key_lower or any(boundary_name.replace('_', ' ') in key_lower for boundary_name in ["very low low", "low medium", "medium high"])):
                 weights_dict[key] = 0.15 # Relevant boundary
            elif "age" in key_lower: # Give some weight to age models
                 weights_dict[key] = 0.10
            elif "verylow" in key_lower: # Give some weight to very low specialized models
                 weights_dict[key] = 0.10
            else:
                weights_dict[key] = 0.05  # Others
            total_weight += weights_dict.get(key, 0) # Use get with 0 default

        # Normalize weights if total_weight is not zero
        if total_weight > 1e-5: # Use a small epsilon to avoid division by near zero
            for key in weights_dict:
                weights_dict[key] /= total_weight
        else:
             # If total weight is zero or near-zero, perhaps assign equal weights or use a default
             num_predictions = len(all_predictions)
             if num_predictions > 0:
                 equal_weight = 1.0 / num_predictions
                 weights_dict = {key: equal_weight for key in all_predictions}
             else:
                 weights_dict = {} # No predictions to weight

        weight_info = "Using estimated weights based on strength range relevance"
        weights_source = 'estimated'

    print(f"\n{weight_info}")
    print("\nApproximate contribution of each model:")

    # Calculate weighted sum
    equation_parts = []
    weighted_sum = 0

    # Sort predictions by estimated contribution
    sorted_predictions = {}
    for key, pred in all_predictions.items():
        # Get weight, defaulting to 0 if key not found in weights (important if weights is empty)
        # Use the weights_dict here
        weight = weights_dict.get(key, 0)
        contribution = pred * weight
        sorted_predictions[key] = (pred, weight, contribution)

    # Sort by contribution
    sorted_contributions = sorted(sorted_predictions.items(),
                                 key=lambda x: abs(x[1][2]),
                                 reverse=True)

    # Display top contributions
    for key, (pred, weight, contribution) in sorted_contributions:
        weighted_sum += contribution
        print(f"  {key}: {pred:.2f} MPa × {weight:.3f} = {contribution:.2f} MPa")
        equation_parts.append(f"({pred:.2f} × {weight:.3f})")

    # Print the equation
    print("\nApproximate ensemble equation:")
    # Filter out parts with zero weight/contribution for a cleaner equation
    non_zero_parts = [part for i, part in enumerate(equation_parts) if abs(sorted_contributions[i][1][2]) > 1e-5] # Use epsilon check
    if non_zero_parts:
         print(f"  {' + '.join(non_zero_parts)} ≈ {weighted_sum:.2f} MPa")
    else:
         print("  No significant contributions identified (all weights near zero).")


    print(f"\nNote: The final prediction ({meta_pred:.2f} MPa) includes additional range-specific adjustments.")

    # Calculate and print key concrete engineering properties
    w_c_ratio = input_data['Water  (component 4)(kg in a m^3 mixture)'].values[0] / input_data['Cement (component 1)(kg in a m^3 mixture)'].values[0]
    total_cementitious = input_data['Cement (component 1)(kg in a m^3 mixture)'].values[0] + input_data['Blast Furnace Slag (component 2)(kg in a m^3 mixture)'].values[0] + input_data['Fly Ash (component 3)(kg in a m^3 mixture)'].values[0]
    # Handle division by zero if total_cementitious is 0 or near 0
    if total_cementitious > 1e-5:
        w_cm_ratio = input_data['Water  (component 4)(kg in a m^3 mixture)'].values[0] / total_cementitious
    else:
        w_cm_ratio = float('inf') # Represent as infinity or a large number if cementitious is zero

    print("\n=== CONCRETE PROPERTIES ===")
    print(f"Water-Cement Ratio: {w_c_ratio:.3f}")
    print(f"Water-Cementitious Materials Ratio: {w_cm_ratio:.3f}")
    print(f"Total Cementitious Content: {total_cementitious:.1f} kg/m³")

    return meta_pred

In [21]:
def test_concrete_mix():
    """
    Test the meta-learner equation with a specific concrete mix.
    """
    # Load the model
    model_path = Path('models/enhanced_catboost_model.joblib')
    model_data = joblib.load(model_path)
    predictor = EnhancedCatBoostPredictor()

    for key, value in model_data.items():
        setattr(predictor, key, value)

    # Fix the generate_meta_features method as in the previous solution
    def fixed_generate_meta_features(predictor, X):
        """Fixed version of generate_meta_features."""
        # Generate predictions from deep CatBoost
        deep_preds = predictor.deep_catboost.predict(X)

        # Initialize the meta_features list
        meta_features = [deep_preds]

        # Generate predictions from range-specific models
        for range_name in predictor.strength_labels:
            if hasattr(predictor, 'range_models') and range_name in predictor.range_models:
                range_preds = predictor.range_models[range_name].predict(X)
                meta_features.append(range_preds)

        # Generate predictions from boundary models if available
        if hasattr(predictor, 'boundary_models') and predictor.boundary_models:
            for name, model in predictor.boundary_models.items():
                boundary_preds = model.predict(X)
                meta_features.append(boundary_preds)

        # Generate predictions from age-specific models if available
        if hasattr(predictor, 'age_models') and predictor.age_models:
            for age_group, model in predictor.age_models.items():
                age_preds = model.predict(X)
                meta_features.append(age_preds)

        # Generate predictions from very low specialized models if available
        if hasattr(predictor, 'very_low_specialized_models') and predictor.very_low_specialized_models:
            for name, model in predictor.very_low_specialized_models.items():
                specialized_preds = model.predict(X)
                meta_features.append(specialized_preds)

        # Generate predictions with medium bias correction if available
        if hasattr(predictor, 'medium_bias_model'):
            # Create bias-corrected predictions
            bias_corrected_preds = deep_preds.copy()

            # Estimate which samples are in medium range
            medium_mask = (deep_preds >= 40) & (deep_preds < 60)

            # For medium range samples, apply bias correction
            if np.any(medium_mask):
                # Get medium range indices
                medium_indices = np.where(medium_mask)[0]

                # Get medium range samples
                X_medium = X.iloc[medium_indices]

                # Get bias predictions for medium range samples
                bias_predictions = predictor.medium_bias_model.predict(X_medium)

                # Apply bias correction (70% of predicted bias)
                for idx, i in enumerate(medium_indices):
                    bias_corrected_preds[i] -= bias_predictions[idx] * 0.7

            meta_features.append(bias_corrected_preds)

        # Determine strength range indicators
        # Since we don't know the true range, we'll estimate based on deep model
        estimated_ranges = pd.cut(
            deep_preds,
            bins=predictor.strength_bins,
            labels=predictor.strength_labels
        )

        # Convert to dummy variables (one-hot encoding)
        range_indicators = pd.get_dummies(pd.Series(estimated_ranges)).values

        # Stack all meta-features
        meta_features_array = np.column_stack(meta_features)

        # Add range indicators and original features
        meta_features_array = np.column_stack([
            meta_features_array,
            range_indicators,
            X.values  # Add original features
        ])

        return meta_features_array

    # Replace the method
    predictor.generate_meta_features = lambda X: fixed_generate_meta_features(predictor, X)


    # Create input data for your test case
    test_data = {
        'Cement (component 1)(kg in a m^3 mixture)': 333,
        'Blast Furnace Slag (component 2)(kg in a m^3 mixture)': 143,
        'Fly Ash (component 3)(kg in a m^3 mixture)': 0,
        'Water  (component 4)(kg in a m^3 mixture)': 230,
        'Superplasticizer (component 5)(kg in a m^3 mixture)': 0,
        'Coarse Aggregate  (component 6)(kg in a m^3 mixture)': 933,
        'Fine Aggregate (component 7)(kg in a m^3 mixture)': 596,
        'Age (day)': 366
    }

    # Convert to DataFrame
    input_df = pd.DataFrame([test_data])

    # Print the equation
    print_meta_learner_equation(predictor, input_df)

# Run the test
test_concrete_mix()


=== META-LEARNER EQUATION BREAKDOWN ===

Deep CatBoost prediction: 40.83 MPa

Range-specific models:
  Very Low range model: 16.49 MPa
  Low range model: 39.11 MPa
  Medium range model: 41.96 MPa
  High range model: 68.45 MPa

Meta-learner raw prediction: 32.76 MPa

=== FINAL META-LEARNER EQUATION ===

For this concrete mix, the model predicts: 32.76 MPa (Low Strength)

Approximating the ensemble equation:

Using saved meta-weights: CatBoost=0.80, MLP=0.20

Approximate contribution of each model:
  Deep CatBoost: 40.83 MPa × 0.000 = 0.00 MPa
  Range-very_low: 16.49 MPa × 0.000 = 0.00 MPa
  Range-low: 39.11 MPa × 0.000 = 0.00 MPa
  Range-medium: 41.96 MPa × 0.000 = 0.00 MPa
  Range-high: 68.45 MPa × 0.000 = 0.00 MPa

Approximate ensemble equation:
  No significant contributions identified (all weights near zero).

Note: The final prediction (32.76 MPa) includes additional range-specific adjustments.

=== CONCRETE PROPERTIES ===
Water-Cement Ratio: 0.691
Water-Cementitious Materials Rat

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Load your results data (assuming you've saved it from the model)
# You can replace this with actual loading from your saved files
# If you don't have the file, create placeholder data
try:
    results_df = pd.read_csv('enhanced_catboost_results.csv')
except FileNotFoundError:
    # Create dummy data if file doesn't exist
    print("Results file not found. Creating placeholder data for visualization.")
    np.random.seed(42)
    n_samples = 206

    actual = np.concatenate([
        np.random.uniform(5, 20, size=int(n_samples*0.19)),  # very_low
        np.random.uniform(20, 40, size=int(n_samples*0.44)),  # low
        np.random.uniform(40, 60, size=int(n_samples*0.28)),  # medium
        np.random.uniform(60, 85, size=n_samples - int(n_samples*0.19) - int(n_samples*0.44) - int(n_samples*0.28))  # high
    ])

    # Base model predictions with some error
    base_pred = actual * np.random.normal(1, 0.11, size=n_samples)
    base_error = np.abs((base_pred - actual) / actual * 100)

    # Ensemble model predictions with less error
    ens_pred = actual * np.random.normal(1, 0.06, size=n_samples)
    ens_error = np.abs((ens_pred - actual) / actual * 100)

    # Assign strength ranges
    ranges = []
    for a in actual:
        if a < 20:
            ranges.append('very_low')
        elif a < 40:
            ranges.append('low')
        elif a < 60:
            ranges.append('medium')
        else:
            ranges.append('high')

    # Create DataFrame
    results_df = pd.DataFrame({
        'Actual_Strength': actual,
        'Deep_CatBoost_Prediction': base_pred,
        'Meta_Learner_Prediction': ens_pred,
        'Strength_Range': ranges,
        'Deep_CatBoost_Error_Pct': base_error,
        'Meta_Learner_Error_Pct': ens_error,
        'Error_Improvement': base_error - ens_error
    })

# Set a consistent style for all plots
# Use a style that's available in current matplotlib
plt.style.use('default')
sns.set_theme(style="whitegrid")
sns.set_palette("colorblind")
colors = {'very_low': '#FF9999', 'low': '#FFCC99', 'medium': '#99CCFF', 'high': '#99FF99'}

# Create figure directory if it doesn't exist
import os
if not os.path.exists('methodology_figures'):
    os.makedirs('methodology_figures')

# 1. FEATURE IMPORTANCE PLOT
def plot_feature_importance():
    # Assuming you have feature importance saved from your model
    # Replace this with loading your actual feature importance data
    feature_importance = pd.DataFrame({
        'Feature': ['very_early_strength', 'water_cementitious_ratio', 'Blast Furnace Slag', 'Water',
                   'high_correction', 'total_cementitious', 'very_low_correction', 'slump_indicator',
                   'maturity_index', 'Fly Ash'],
        'Importance': [22.02, 13.58, 4.19, 4.04, 3.38, 3.28, 3.23, 3.19, 3.17, 2.86]
    })

    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title('Top 10 Features by Importance', fontsize=16)
    plt.xlabel('Importance Score', fontsize=14)
    plt.ylabel('Feature', fontsize=14)
    plt.tight_layout()
    plt.savefig('methodology_figures/feature_importance.png', dpi=300)
    plt.close()
    print("Created: Feature Importance Plot")

# 2. PREDICTED VS ACTUAL PLOT WITH ERROR BANDS
def plot_predicted_vs_actual():
    # Create figure
    plt.figure(figsize=(12, 8))

    # Scatter plot - colored by strength range
    for range_name in ['very_low', 'low', 'medium', 'high']:
        range_data = results_df[results_df['Strength_Range'] == range_name]
        plt.scatter(range_data['Actual_Strength'],
                   range_data['Meta_Learner_Prediction'],
                   alpha=0.7, label=range_name.replace('_', ' ').title(),
                   color=colors[range_name], s=60)

    # Perfect prediction line
    max_val = max(results_df['Actual_Strength'].max(), results_df['Meta_Learner_Prediction'].max())
    plt.plot([0, max_val], [0, max_val], 'k--', label='Perfect Prediction')

    # 10% error bands
    x = np.linspace(0, max_val, 100)
    plt.fill_between(x, x*0.9, x*1.1, alpha=0.1, color='gray', label='±10% Error Band')

    plt.title('Predicted vs Actual Concrete Strength', fontsize=16)
    plt.xlabel('Actual Strength (MPa)', fontsize=14)
    plt.ylabel('Predicted Strength (MPa)', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('methodology_figures/predicted_vs_actual.png', dpi=300)
    plt.close()
    print("Created: Predicted vs Actual Plot")

# 3. ERROR DISTRIBUTION BY STRENGTH RANGE
def plot_error_distribution():
    # Calculate percent errors for both models
    results_df['Base_Percent_Error'] = results_df['Deep_CatBoost_Error_Pct']
    results_df['Ensemble_Percent_Error'] = results_df['Meta_Learner_Error_Pct']

    # Prepare data for plotting
    error_data = []
    for range_name in ['very_low', 'low', 'medium', 'high']:
        range_data = results_df[results_df['Strength_Range'] == range_name]

        # Base model errors
        error_data.append({
            'Strength Range': range_name.replace('_', ' ').title(),
            'Error (%)': range_data['Base_Percent_Error'].mean(),
            'Model': 'Base Model'
        })

        # Ensemble model errors
        error_data.append({
            'Strength Range': range_name.replace('_', ' ').title(),
            'Error (%)': range_data['Ensemble_Percent_Error'].mean(),
            'Model': 'Ensemble Model'
        })

    error_df = pd.DataFrame(error_data)

    # Create plot
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Strength Range', y='Error (%)',
                hue='Model', data=error_df, palette=['#4472c4', '#70ad47'])

    plt.title('Mean Percentage Error by Strength Range', fontsize=16)
    plt.xlabel('Concrete Strength Range', fontsize=14)
    plt.ylabel('Mean Percentage Error (%)', fontsize=14)
    plt.legend(title='', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('methodology_figures/error_by_range.png', dpi=300)
    plt.close()
    print("Created: Error Distribution Plot")

# 4. MODEL PERFORMANCE COMPARISON
def plot_performance_metrics():
    # Calculate metrics from results if available
    try:
        base_r2 = r2_score(results_df['Actual_Strength'], results_df['Deep_CatBoost_Prediction'])
        base_rmse = np.sqrt(mean_squared_error(results_df['Actual_Strength'], results_df['Deep_CatBoost_Prediction']))
        base_mae = mean_absolute_error(results_df['Actual_Strength'], results_df['Deep_CatBoost_Prediction'])
        base_within_5 = (results_df['Deep_CatBoost_Error_Pct'] <= 5).mean() * 100
        base_within_10 = (results_df['Deep_CatBoost_Error_Pct'] <= 10).mean() * 100

        ens_r2 = r2_score(results_df['Actual_Strength'], results_df['Meta_Learner_Prediction'])
        ens_rmse = np.sqrt(mean_squared_error(results_df['Actual_Strength'], results_df['Meta_Learner_Prediction']))
        ens_mae = mean_absolute_error(results_df['Actual_Strength'], results_df['Meta_Learner_Prediction'])
        ens_within_5 = (results_df['Meta_Learner_Error_Pct'] <= 5).mean() * 100
        ens_within_10 = (results_df['Meta_Learner_Error_Pct'] <= 10).mean() * 100
    except:
        # Use values from the original code
        base_r2, base_rmse, base_mae, base_within_5, base_within_10 = 0.945, 3.99, 2.54, 48.06, 74.76
        ens_r2, ens_rmse, ens_mae, ens_within_5, ens_within_10 = 0.977, 2.61, 1.47, 69.42, 88.35

    # Prepare data
    metrics = pd.DataFrame({
        'Metric': ['R²', 'RMSE (MPa)', 'MAE (MPa)', 'Within 5%', 'Within 10%'],
        'Base Model': [base_r2, base_rmse, base_mae, base_within_5, base_within_10],
        'Ensemble Model': [ens_r2, ens_rmse, ens_mae, ens_within_5, ens_within_10]
    })

    # Convert to long format for seaborn
    metrics_long = pd.melt(metrics, id_vars=['Metric'],
                           var_name='Model', value_name='Value')

    # Create plot
    plt.figure(figsize=(12, 8))

    # For barplot, we need to handle the metrics separately because they have different scales
    fig, axes = plt.subplots(1, 5, figsize=(15, 6), sharey=False)

    # Define color palette
    palette = {'Base Model': '#4472c4', 'Ensemble Model': '#70ad47'}

    # Plot each metric in its own subplot
    for i, metric in enumerate(metrics['Metric'].unique()):
        metric_data = metrics_long[metrics_long['Metric'] == metric]

        sns.barplot(x='Model', y='Value', data=metric_data, ax=axes[i], palette=palette)
        axes[i].set_title(metric)
        axes[i].set_xlabel('')

        # Add value labels on bars
        for j, p in enumerate(axes[i].patches):
            height = p.get_height()
            if metric == 'R²':
                axes[i].text(p.get_x() + p.get_width()/2., height + 0.01, f'{height:.3f}',
                            ha="center", va="bottom")
            else:
                axes[i].text(p.get_x() + p.get_width()/2., height + 0.01, f'{height:.1f}',
                            ha="center", va="bottom")

    plt.suptitle('Performance Comparison: Base vs Ensemble Model', fontsize=16)
    plt.tight_layout()
    plt.savefig('methodology_figures/performance_comparison.png', dpi=300)
    plt.close()
    print("Created: Performance Metrics Comparison Plot")

# 5. IMPROVEMENT BY STRENGTH RANGE
def plot_improvement_by_range():
    # Calculate performance metrics by strength range
    range_improvement = []

    for range_name in ['very_low', 'low', 'medium', 'high']:
        range_data = results_df[results_df['Strength_Range'] == range_name]

        # Base model - within 10%
        base_within_10 = (range_data['Deep_CatBoost_Error_Pct'] <= 10).mean() * 100

        # Ensemble model - within 10%
        ensemble_within_10 = (range_data['Meta_Learner_Error_Pct'] <= 10).mean() * 100

        # Improvement percentage points
        improvement = ensemble_within_10 - base_within_10

        range_improvement.append({
            'Strength Range': range_name.replace('_', ' ').title(),
            'Base Model': base_within_10,
            'Ensemble Model': ensemble_within_10,
            'Improvement': improvement
        })

    improve_df = pd.DataFrame(range_improvement)

    # Create plot
    fig, ax1 = plt.subplots(figsize=(12, 8))

    # Bar chart for base and ensemble
    x = np.arange(len(improve_df['Strength Range']))
    width = 0.35

    ax1.bar(x - width/2, improve_df['Base Model'], width, label='Base Model', color='#4472c4')
    ax1.bar(x + width/2, improve_df['Ensemble Model'], width, label='Ensemble Model', color='#70ad47')

    # Line chart for improvement
    ax2 = ax1.twinx()
    ax2.plot(x, improve_df['Improvement'], 'ro-', linewidth=2, label='Improvement')

    # Add data labels for improvement
    for i, val in enumerate(improve_df['Improvement']):
        ax2.annotate(f'{val:.1f}pp', xy=(i, val), xytext=(0, 5),
                    textcoords='offset points', ha='center', fontsize=10, color='red')

    # Customize plot
    ax1.set_xlabel('Strength Range', fontsize=14)
    ax1.set_ylabel('Predictions Within 10% (%)', fontsize=14)
    ax2.set_ylabel('Improvement (percentage points)', fontsize=14, color='red')

    ax1.set_xticks(x)
    ax1.set_xticklabels(improve_df['Strength Range'])

    ax1.legend(loc='upper left')
    ax2.legend(loc='lower right')

    plt.title('Model Improvement by Strength Range', fontsize=16)
    plt.tight_layout()
    plt.savefig('methodology_figures/improvement_by_range.png', dpi=300)
    plt.close()
    print("Created: Improvement by Strength Range Plot")

# 6. ERROR ANALYSIS VISUALIZATION
def plot_error_analysis():
    # Instead of trying to load the error analysis, we'll create synthetic data
    # based on the values mentioned in the code

    # Creating a figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

    # Plot 1: Error rate by strength range
    strength_error_rates = {
        'Very Low': 28.2,
        'Low': 8.8,
        'Medium': 5.3,
        'High': 10.5
    }

    ax1.bar(strength_error_rates.keys(), strength_error_rates.values(),
           color=[colors['very_low'], colors['low'], colors['medium'], colors['high']])

    ax1.set_title('Error Rate by Strength Range', fontsize=14)
    ax1.set_xlabel('Strength Range', fontsize=12)
    ax1.set_ylabel('Error Rate (%)', fontsize=12)
    ax1.grid(axis='y', alpha=0.3)

    # Plot 2: Error correlation with features
    error_correlations = {
        'late_age_factor': -0.252,
        'log_age': -0.249,
        'sqrt_age': -0.234,
        'maturity_index': -0.190,
        'water_excess_indicator': 0.190
    }

    features = list(error_correlations.keys())
    values = list(error_correlations.values())
    bar_colors = ['#4c78a8' if v < 0 else '#72b7b2' for v in values]

    # Sort by absolute value
    sorted_indices = np.argsort(np.abs(values))[::-1]
    sorted_features = [features[i] for i in sorted_indices]
    sorted_values = [values[i] for i in sorted_indices]
    sorted_colors = [bar_colors[i] for i in sorted_indices]

    ax2.barh(sorted_features, sorted_values, color=sorted_colors)

    ax2.set_title('Top 5 Features Correlated with Error', fontsize=14)
    ax2.set_xlabel('Correlation Coefficient', fontsize=12)
    ax2.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    ax2.grid(axis='x', alpha=0.3)

    plt.tight_layout()
    plt.savefig('methodology_figures/error_analysis.png', dpi=300)
    plt.close()
    print("Created: Error Analysis Plot")

# 7. MODEL ENSEMBLE DIAGRAM
def create_ensemble_diagram():
    """
    Create a visual representation of the ensemble structure.
    """
    plt.figure(figsize=(12, 10))

    # Set up coordinates
    y_pos = {
        'input': 0.9,
        'base': 0.75,
        'specialized': 0.5,
        'meta': 0.25,
        'output': 0.1
    }

    # Plot components
    plt.scatter(0.5, y_pos['input'], s=300, color='#4472c4', zorder=5)
    plt.scatter(0.5, y_pos['base'], s=500, color='#4472c4', zorder=5)

    specialized_x = [0.2, 0.4, 0.6, 0.8]
    specialized_colors = [colors['very_low'], colors['low'], colors['medium'], colors['high']]

    for i, x in enumerate(specialized_x):
        plt.scatter(x, y_pos['specialized'], s=400, color=specialized_colors[i], zorder=5)

    plt.scatter(0.5, y_pos['meta'], s=500, color='#70ad47', zorder=5)
    plt.scatter(0.5, y_pos['output'], s=300, color='#70ad47', zorder=5)

    # Add connecting lines
    plt.plot([0.5, 0.5], [y_pos['input'], y_pos['base']], 'k-', linewidth=2)

    for x in specialized_x:
        plt.plot([0.5, x], [y_pos['base'], y_pos['specialized']], 'k-', linewidth=2)
        plt.plot([x, 0.5], [y_pos['specialized'], y_pos['meta']], 'k-', linewidth=2)

    plt.plot([0.5, 0.5], [y_pos['meta'], y_pos['output']], 'k-', linewidth=2)

    # Add labels
    plt.text(0.5, y_pos['input']+0.05, "Input Features", ha='center', fontsize=12)
    plt.text(0.5, y_pos['base']+0.05, "Deep CatBoost Base Model", ha='center', fontsize=12)

    specialized_labels = ["Very Low\nRange Model", "Low\nRange Model",
                         "Medium\nRange Model", "High Range &\nBoundary Models"]

    for i, x in enumerate(specialized_x):
        plt.text(x, y_pos['specialized']+0.05, specialized_labels[i], ha='center', fontsize=10)

    plt.text(0.5, y_pos['meta']+0.05, "Meta-Learner Ensemble", ha='center', fontsize=12)
    plt.text(0.5, y_pos['output']+0.05, "Final Prediction", ha='center', fontsize=12)

    # Remove axes
    plt.axis('off')

    # Set title
    plt.title('Model Ensemble Structure', fontsize=16)

    plt.tight_layout()
    plt.savefig('methodology_figures/ensemble_diagram.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Created: Model Ensemble Diagram")

# Generate all plots
if __name__ == "__main__":
    plot_feature_importance()
    plot_predicted_vs_actual()
    plot_error_distribution()
    plot_performance_metrics()
    plot_improvement_by_range()
    plot_error_analysis()
    create_ensemble_diagram()

    print("\nAll methodology plots have been generated and saved to 'methodology_figures/' directory")

Created: Feature Importance Plot
Created: Predicted vs Actual Plot
Created: Error Distribution Plot
Created: Performance Metrics Comparison Plot
Created: Improvement by Strength Range Plot
Created: Error Analysis Plot
Created: Model Ensemble Diagram

All methodology plots have been generated and saved to 'methodology_figures/' directory


<Figure size 1200x800 with 0 Axes>

In [29]:
!zip -r /content/project.zip /content/ -x '/content/sample_data*'

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.08.13/ (stored 0%)
  adding: content/.config/logs/2025.08.13/13.41.05.763189.log (deflated 92%)
  adding: content/.config/logs/2025.08.13/13.41.34.546254.log (deflated 58%)
  adding: content/.config/logs/2025.08.13/13.41.58.444013.log (deflated 58%)
  adding: content/.config/logs/2025.08.13/13.41.59.127735.log (deflated 56%)
  adding: content/.config/logs/2025.08.13/13.41.43.478751.log (deflated 86%)
  adding: content/.config/logs/2025.08.13/13.41.49.317369.log (deflated 58%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/config_sentinel (stored 0%)
  